# Interpretations using LIME

# Heart dataset

In [1]:
import pandas as pd
import numpy as np
from datetime import timedelta, date
import datetime as dt
import sys
sys.path.append('./explanation_helper')
import expla_helper as explanation

heart = pd.read_csv('data/datasets_33180_43520_heart.csv', sep = ',')
heart = explanation.to_float(heart)

print(heart.dtypes)

age         float64
sex         float64
cp          float64
trestbps    float64
chol        float64
fbs         float64
restecg     float64
thalach     float64
exang       float64
oldpeak     float64
slope       float64
ca          float64
thal        float64
target        int64
dtype: object


# Wine dataset

In [2]:
import pandas as pd
import numpy as np
from datetime import timedelta, date
import datetime as dt
import sys
sys.path.append('./explanation_helper')
import expla_helper as explanation


wine = pd.read_csv('data/wine_limpo.csv', sep = ',')
# wine.drop(['Unnamed: 0'], axis = 1, inplace = True)

wine = explanation.to_float(wine)

print(wine.dtypes)

fixed acidity           float64
volatile acidity        float64
citric acid             float64
residual sugar          float64
chlorides               float64
free sulfur dioxide     float64
total sulfur dioxide    float64
density                 float64
pH                      float64
sulphates               float64
alcohol                 float64
target                    int64
dtype: object


# Diabetes dataset

In [3]:
import pandas as pd
import numpy as np
from datetime import timedelta, date
import datetime as dt

import sys
sys.path.append('./explanation_helper')
import expla_helper as explanation


diabetes = pd.read_csv('data/diabetes_limpo.csv', sep = ',')
# diabetes.drop(['Unnamed: 0'], axis = 1, inplace = True)

diabetes = explanation.to_float(diabetes)

print(diabetes.dtypes)


Pregnancies                 float64
Glucose                     float64
BloodPressure               float64
SkinThickness               float64
Insulin                     float64
BMI                         float64
DiabetesPedigreeFunction    float64
Age                         float64
target                        int64
dtype: object


# LIME 

## Choosing 4 instances (TP, FP, TN, FN) of each dataset randomly from DT to be our baseline

Code to choose 4 instances (TP, FP, TN, FN) of each dataset randomly, we have choosed the decision tree as the baseline classificator to choose the confusion matrix metrics

### Get the samples from the baseline

In [4]:
import numpy as np
import pandas as pd
import lime
import lime.lime_tabular
import sys
sys.path.append('./ml_helper')
import ml_helper_train_model as model_train
import ml_helper_serialize_model as serializer
sys.path.append('./explanation_helper')
import expla_helper as explanation
from sklearn.model_selection import train_test_split

np.random.seed(1)

heart = pd.read_csv('data/datasets_33180_43520_heart.csv', sep = ',')
model = serializer.load_model(file_name = 'serialized_model/DT_heart.pkl')
heart_sample = explanation.sample_baseline_model(model, heart)

wine = pd.read_csv('data/wine_limpo.csv', sep = ',')
model = serializer.load_model(file_name = 'serialized_model/DT_wine2.pkl')
wine_sample = explanation.sample_baseline_model(model, wine)

diabetes = pd.read_csv('data/diabetes_limpo.csv', sep = ',')
model = serializer.load_model(file_name = 'serialized_model/DT_diabetes2.pkl')
diabetes_sample = explanation.sample_baseline_model(model, diabetes)

# print(heart)
print(heart_sample)
print(wine_sample)
print(diabetes_sample)

     real_label  pred_label mat_conf_mod
143           1           0           FN
203           0           1           FP
242           0           0           TN
96            1           1           TP
      real_label  pred_label mat_conf_mod
86             1           0           FN
1458           0           1           FP
791            0           0           TN
1003           1           1           TP
     real_label  pred_label mat_conf_mod
291           1           0           FN
725           0           1           FP
60            0           0           TN
755           1           1           TP
Int64Index([143, 203, 242, 96], dtype='int64')
Int64Index([86, 1458, 791, 1003], dtype='int64')
Int64Index([291, 725, 60, 755], dtype='int64')


# Parâmetros LIME

In [6]:
import pandas as pd
import numpy as np
import lime
import lime.lime_tabular
import sys
sys.path.append('./ml_helper')
import ml_helper_train_model as model_train
import ml_helper_serialize_model as serializer
from sklearn.model_selection import train_test_split
import math

feature_selection = ['forward_selection', 'lasso_path', 'none', 'auto']
discretize_continuous = [True]
discretizer = ['quartile', 'decile']
sample_around_instance = [True, False]

## MLP

### Heart dataset - explanation

In [7]:
# import numpy as np
import lime
import lime.lime_tabular
import sys
sys.path.append('./ml_helper')
import ml_helper_train_model as model_train
import ml_helper_serialize_model as serializer
sys.path.append('./explanation_helper')
import expla_helper as explanation
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import math

heart = pd.read_csv('data/datasets_33180_43520_heart.csv', sep = ',')
heart = explanation.to_float(heart)

model_file_name = 'serialized_model/MLP_heart.pkl'
model = serializer.load_model(model_file_name)

dataset = heart.copy(deep = True)
target_names = np.array(['saudável', 'doente'])

folder = 'serialized_explanation/'
model_dataset = model_file_name.replace('serialized_model/','').replace('.pkl', '').replace('_', '#')

kernel_width = [math.ceil(dataset.shape[1]*0.1),
                math.ceil(dataset.shape[1]*0.3),
                math.ceil(dataset.shape[1]*0.5),
                math.ceil(dataset.shape[1]*0.7)]

for size in kernel_width:
    for sample in heart_sample:
        for f_selec in feature_selection:
            for disc in discretizer:
                for sai in sample_around_instance:
                    instance_index = sample
                    num_features = dataset.columns.size - 1 #removing the target

                    path = """{0}{1}#instance:{2}#""".format(folder, model_dataset, instance_index)
                    params_01 = """num_features:{0}#feat_selec:{1}#discre:{2}#""".format(num_features, f_selec, disc)
                    params_02 = """sai:{0}#kernel_s:{1}.html""".format(sai, size)
                    
                    final_file = """{0}{1}{2}""".format(path, params_01, params_02)
                    print(final_file)

                    explanation.explain_it_param(model,
                                               dataset,
                                               target_names,
                                               num_features,
                                               instance_index,
                                               final_file,
                                               #object level
                                               size,
                                               f_selec,     
                                               discretize_continuous,
                                               disc,
                                               sai)


serialized_explanation/MLP#heart#instance:143#num_features:13#feat_selec:forward_selection#discre:quartile#sai:True#kernel_s:2.html
serialized_explanation/MLP#heart#instance:143#num_features:13#feat_selec:forward_selection#discre:quartile#sai:False#kernel_s:2.html
serialized_explanation/MLP#heart#instance:143#num_features:13#feat_selec:forward_selection#discre:decile#sai:True#kernel_s:2.html
serialized_explanation/MLP#heart#instance:143#num_features:13#feat_selec:forward_selection#discre:decile#sai:False#kernel_s:2.html
serialized_explanation/MLP#heart#instance:143#num_features:13#feat_selec:lasso_path#discre:quartile#sai:True#kernel_s:2.html
serialized_explanation/MLP#heart#instance:143#num_features:13#feat_selec:lasso_path#discre:quartile#sai:False#kernel_s:2.html
serialized_explanation/MLP#heart#instance:143#num_features:13#feat_selec:lasso_path#discre:decile#sai:True#kernel_s:2.html
serialized_explanation/MLP#heart#instance:143#num_features:13#feat_selec:lasso_path#discre:decile#sa

serialized_explanation/MLP#heart#instance:143#num_features:13#feat_selec:forward_selection#discre:decile#sai:False#kernel_s:5.html
serialized_explanation/MLP#heart#instance:143#num_features:13#feat_selec:lasso_path#discre:quartile#sai:True#kernel_s:5.html
serialized_explanation/MLP#heart#instance:143#num_features:13#feat_selec:lasso_path#discre:quartile#sai:False#kernel_s:5.html
serialized_explanation/MLP#heart#instance:143#num_features:13#feat_selec:lasso_path#discre:decile#sai:True#kernel_s:5.html
serialized_explanation/MLP#heart#instance:143#num_features:13#feat_selec:lasso_path#discre:decile#sai:False#kernel_s:5.html
serialized_explanation/MLP#heart#instance:143#num_features:13#feat_selec:none#discre:quartile#sai:True#kernel_s:5.html
serialized_explanation/MLP#heart#instance:143#num_features:13#feat_selec:none#discre:quartile#sai:False#kernel_s:5.html
serialized_explanation/MLP#heart#instance:143#num_features:13#feat_selec:none#discre:decile#sai:True#kernel_s:5.html
serialized_expl

serialized_explanation/MLP#heart#instance:143#num_features:13#feat_selec:lasso_path#discre:decile#sai:True#kernel_s:7.html
serialized_explanation/MLP#heart#instance:143#num_features:13#feat_selec:lasso_path#discre:decile#sai:False#kernel_s:7.html
serialized_explanation/MLP#heart#instance:143#num_features:13#feat_selec:none#discre:quartile#sai:True#kernel_s:7.html
serialized_explanation/MLP#heart#instance:143#num_features:13#feat_selec:none#discre:quartile#sai:False#kernel_s:7.html
serialized_explanation/MLP#heart#instance:143#num_features:13#feat_selec:none#discre:decile#sai:True#kernel_s:7.html
serialized_explanation/MLP#heart#instance:143#num_features:13#feat_selec:none#discre:decile#sai:False#kernel_s:7.html
serialized_explanation/MLP#heart#instance:143#num_features:13#feat_selec:auto#discre:quartile#sai:True#kernel_s:7.html
serialized_explanation/MLP#heart#instance:143#num_features:13#feat_selec:auto#discre:quartile#sai:False#kernel_s:7.html
serialized_explanation/MLP#heart#instanc

serialized_explanation/MLP#heart#instance:143#num_features:13#feat_selec:none#discre:quartile#sai:False#kernel_s:10.html
serialized_explanation/MLP#heart#instance:143#num_features:13#feat_selec:none#discre:decile#sai:True#kernel_s:10.html
serialized_explanation/MLP#heart#instance:143#num_features:13#feat_selec:none#discre:decile#sai:False#kernel_s:10.html
serialized_explanation/MLP#heart#instance:143#num_features:13#feat_selec:auto#discre:quartile#sai:True#kernel_s:10.html
serialized_explanation/MLP#heart#instance:143#num_features:13#feat_selec:auto#discre:quartile#sai:False#kernel_s:10.html
serialized_explanation/MLP#heart#instance:143#num_features:13#feat_selec:auto#discre:decile#sai:True#kernel_s:10.html
serialized_explanation/MLP#heart#instance:143#num_features:13#feat_selec:auto#discre:decile#sai:False#kernel_s:10.html
serialized_explanation/MLP#heart#instance:203#num_features:13#feat_selec:forward_selection#discre:quartile#sai:True#kernel_s:10.html
serialized_explanation/MLP#hear

### Wine dataset - explanation

In [8]:
# import numpy as np
import lime
import lime.lime_tabular
import sys
sys.path.append('./ml_helper')
import ml_helper_train_model as model_train
import ml_helper_serialize_model as serializer
sys.path.append('./explanation_helper')
import expla_helper as explanation
from sklearn.model_selection import train_test_split

wine = pd.read_csv('data/wine_limpo.csv', sep = ',')
# wine.drop(['Unnamed: 0'], axis = 1, inplace = True)
wine = explanation.to_float(wine)

model_file_name = 'serialized_model/MLP_wine2.pkl'
model = serializer.load_model(model_file_name)

dataset = wine.copy(deep = True)
target_names = np.array(['ruim', 'bom'])

folder = 'serialized_explanation/'
model_dataset = model_file_name.replace('serialized_model/','').replace('.pkl', '').replace('_', '#')

kernel_width = [math.ceil(dataset.shape[1]*0.1),
                math.ceil(dataset.shape[1]*0.3),
                math.ceil(dataset.shape[1]*0.5),
                math.ceil(dataset.shape[1]*0.7)]

for size in kernel_width:
    for sample in wine_sample:
        for f_selec in feature_selection:
            for disc in discretizer:
                for sai in sample_around_instance:
                    instance_index = sample
                    num_features = dataset.columns.size - 1 #removing the target

                    path = """{0}{1}#instance:{2}#""".format(folder, model_dataset, instance_index)
                    params_01 = """num_features:{0}#feat_selec:{1}#discre:{2}#""".format(num_features, f_selec, disc)
                    params_02 = """sai:{0}#kernel_s:{1}.html""".format(sai, size)
                    
                    final_file = """{0}{1}{2}""".format(path, params_01, params_02)
                    print(final_file)

                    explanation.explain_it_param(model,
                                               dataset,
                                               target_names,
                                               num_features,
                                               instance_index,
                                               final_file,
                                               #object level
                                               size,
                                               f_selec,     
                                               discretize_continuous,
                                               disc,
                                               sai)

serialized_explanation/MLP#wine2#instance:86#num_features:11#feat_selec:forward_selection#discre:quartile#sai:True#kernel_s:2.html
serialized_explanation/MLP#wine2#instance:86#num_features:11#feat_selec:forward_selection#discre:quartile#sai:False#kernel_s:2.html
serialized_explanation/MLP#wine2#instance:86#num_features:11#feat_selec:forward_selection#discre:decile#sai:True#kernel_s:2.html
serialized_explanation/MLP#wine2#instance:86#num_features:11#feat_selec:forward_selection#discre:decile#sai:False#kernel_s:2.html
serialized_explanation/MLP#wine2#instance:86#num_features:11#feat_selec:lasso_path#discre:quartile#sai:True#kernel_s:2.html
serialized_explanation/MLP#wine2#instance:86#num_features:11#feat_selec:lasso_path#discre:quartile#sai:False#kernel_s:2.html
serialized_explanation/MLP#wine2#instance:86#num_features:11#feat_selec:lasso_path#discre:decile#sai:True#kernel_s:2.html
serialized_explanation/MLP#wine2#instance:86#num_features:11#feat_selec:lasso_path#discre:decile#sai:False#

serialized_explanation/MLP#wine2#instance:86#num_features:11#feat_selec:forward_selection#discre:decile#sai:False#kernel_s:4.html
serialized_explanation/MLP#wine2#instance:86#num_features:11#feat_selec:lasso_path#discre:quartile#sai:True#kernel_s:4.html
serialized_explanation/MLP#wine2#instance:86#num_features:11#feat_selec:lasso_path#discre:quartile#sai:False#kernel_s:4.html
serialized_explanation/MLP#wine2#instance:86#num_features:11#feat_selec:lasso_path#discre:decile#sai:True#kernel_s:4.html
serialized_explanation/MLP#wine2#instance:86#num_features:11#feat_selec:lasso_path#discre:decile#sai:False#kernel_s:4.html
serialized_explanation/MLP#wine2#instance:86#num_features:11#feat_selec:none#discre:quartile#sai:True#kernel_s:4.html
serialized_explanation/MLP#wine2#instance:86#num_features:11#feat_selec:none#discre:quartile#sai:False#kernel_s:4.html
serialized_explanation/MLP#wine2#instance:86#num_features:11#feat_selec:none#discre:decile#sai:True#kernel_s:4.html
serialized_explanation/

serialized_explanation/MLP#wine2#instance:86#num_features:11#feat_selec:lasso_path#discre:decile#sai:True#kernel_s:6.html
serialized_explanation/MLP#wine2#instance:86#num_features:11#feat_selec:lasso_path#discre:decile#sai:False#kernel_s:6.html
serialized_explanation/MLP#wine2#instance:86#num_features:11#feat_selec:none#discre:quartile#sai:True#kernel_s:6.html
serialized_explanation/MLP#wine2#instance:86#num_features:11#feat_selec:none#discre:quartile#sai:False#kernel_s:6.html
serialized_explanation/MLP#wine2#instance:86#num_features:11#feat_selec:none#discre:decile#sai:True#kernel_s:6.html
serialized_explanation/MLP#wine2#instance:86#num_features:11#feat_selec:none#discre:decile#sai:False#kernel_s:6.html
serialized_explanation/MLP#wine2#instance:86#num_features:11#feat_selec:auto#discre:quartile#sai:True#kernel_s:6.html
serialized_explanation/MLP#wine2#instance:86#num_features:11#feat_selec:auto#discre:quartile#sai:False#kernel_s:6.html
serialized_explanation/MLP#wine2#instance:86#num

serialized_explanation/MLP#wine2#instance:86#num_features:11#feat_selec:none#discre:quartile#sai:False#kernel_s:9.html
serialized_explanation/MLP#wine2#instance:86#num_features:11#feat_selec:none#discre:decile#sai:True#kernel_s:9.html
serialized_explanation/MLP#wine2#instance:86#num_features:11#feat_selec:none#discre:decile#sai:False#kernel_s:9.html
serialized_explanation/MLP#wine2#instance:86#num_features:11#feat_selec:auto#discre:quartile#sai:True#kernel_s:9.html
serialized_explanation/MLP#wine2#instance:86#num_features:11#feat_selec:auto#discre:quartile#sai:False#kernel_s:9.html
serialized_explanation/MLP#wine2#instance:86#num_features:11#feat_selec:auto#discre:decile#sai:True#kernel_s:9.html
serialized_explanation/MLP#wine2#instance:86#num_features:11#feat_selec:auto#discre:decile#sai:False#kernel_s:9.html
serialized_explanation/MLP#wine2#instance:1458#num_features:11#feat_selec:forward_selection#discre:quartile#sai:True#kernel_s:9.html
serialized_explanation/MLP#wine2#instance:145

### Diabetes dataset

In [9]:
# import numpy as np
import lime
import lime.lime_tabular
import sys
sys.path.append('./ml_helper')
import ml_helper_train_model as model_train
import ml_helper_serialize_model as serializer
sys.path.append('./explanation_helper')
import expla_helper as explanation
from sklearn.model_selection import train_test_split

diabetes = pd.read_csv('data/diabetes_limpo.csv', sep = ',')
# diabetes.drop(['Unnamed: 0'], axis = 1, inplace = True)
diabetes = explanation.to_float(diabetes)

model_file_name = 'serialized_model/MLP_diabetes2.pkl'
model = serializer.load_model(model_file_name)

dataset = diabetes.copy(deep = True)
target_names = np.array(['saudável', 'doente'])

folder = 'serialized_explanation/'
model_dataset = model_file_name.replace('serialized_model/','').replace('.pkl', '').replace('_', '#')


kernel_width = [math.ceil(dataset.shape[1]*0.1),
                math.ceil(dataset.shape[1]*0.3),
                math.ceil(dataset.shape[1]*0.5),
                math.ceil(dataset.shape[1]*0.7)]

for size in kernel_width:
    for sample in diabetes_sample:
        for f_selec in feature_selection:
            for disc in discretizer:
                for sai in sample_around_instance:
                    instance_index = sample
                    num_features = dataset.columns.size - 1 #removing the target

                    path = """{0}{1}#instance:{2}#""".format(folder, model_dataset, instance_index)
                    params_01 = """num_features:{0}#feat_selec:{1}#discre:{2}#""".format(num_features, f_selec, disc)
                    params_02 = """sai:{0}#kernel_s:{1}.html""".format(sai, size)
                    
                    final_file = """{0}{1}{2}""".format(path, params_01, params_02)
                    print(final_file)

                    explanation.explain_it_param(model,
                                               dataset,
                                               target_names,
                                               num_features,
                                               instance_index,
                                               final_file,
                                               #object level
                                               size,
                                               f_selec,     
                                               discretize_continuous,
                                               disc,
                                               sai)


serialized_explanation/MLP#diabetes2#instance:291#num_features:8#feat_selec:forward_selection#discre:quartile#sai:True#kernel_s:1.html
serialized_explanation/MLP#diabetes2#instance:291#num_features:8#feat_selec:forward_selection#discre:quartile#sai:False#kernel_s:1.html
serialized_explanation/MLP#diabetes2#instance:291#num_features:8#feat_selec:forward_selection#discre:decile#sai:True#kernel_s:1.html
serialized_explanation/MLP#diabetes2#instance:291#num_features:8#feat_selec:forward_selection#discre:decile#sai:False#kernel_s:1.html
serialized_explanation/MLP#diabetes2#instance:291#num_features:8#feat_selec:lasso_path#discre:quartile#sai:True#kernel_s:1.html
serialized_explanation/MLP#diabetes2#instance:291#num_features:8#feat_selec:lasso_path#discre:quartile#sai:False#kernel_s:1.html
serialized_explanation/MLP#diabetes2#instance:291#num_features:8#feat_selec:lasso_path#discre:decile#sai:True#kernel_s:1.html
serialized_explanation/MLP#diabetes2#instance:291#num_features:8#feat_selec:las

serialized_explanation/MLP#diabetes2#instance:291#num_features:8#feat_selec:forward_selection#discre:quartile#sai:False#kernel_s:3.html
serialized_explanation/MLP#diabetes2#instance:291#num_features:8#feat_selec:forward_selection#discre:decile#sai:True#kernel_s:3.html
serialized_explanation/MLP#diabetes2#instance:291#num_features:8#feat_selec:forward_selection#discre:decile#sai:False#kernel_s:3.html
serialized_explanation/MLP#diabetes2#instance:291#num_features:8#feat_selec:lasso_path#discre:quartile#sai:True#kernel_s:3.html
serialized_explanation/MLP#diabetes2#instance:291#num_features:8#feat_selec:lasso_path#discre:quartile#sai:False#kernel_s:3.html
serialized_explanation/MLP#diabetes2#instance:291#num_features:8#feat_selec:lasso_path#discre:decile#sai:True#kernel_s:3.html
serialized_explanation/MLP#diabetes2#instance:291#num_features:8#feat_selec:lasso_path#discre:decile#sai:False#kernel_s:3.html
serialized_explanation/MLP#diabetes2#instance:291#num_features:8#feat_selec:none#discre

serialized_explanation/MLP#diabetes2#instance:291#num_features:8#feat_selec:forward_selection#discre:decile#sai:True#kernel_s:5.html
serialized_explanation/MLP#diabetes2#instance:291#num_features:8#feat_selec:forward_selection#discre:decile#sai:False#kernel_s:5.html
serialized_explanation/MLP#diabetes2#instance:291#num_features:8#feat_selec:lasso_path#discre:quartile#sai:True#kernel_s:5.html
serialized_explanation/MLP#diabetes2#instance:291#num_features:8#feat_selec:lasso_path#discre:quartile#sai:False#kernel_s:5.html
serialized_explanation/MLP#diabetes2#instance:291#num_features:8#feat_selec:lasso_path#discre:decile#sai:True#kernel_s:5.html
serialized_explanation/MLP#diabetes2#instance:291#num_features:8#feat_selec:lasso_path#discre:decile#sai:False#kernel_s:5.html
serialized_explanation/MLP#diabetes2#instance:291#num_features:8#feat_selec:none#discre:quartile#sai:True#kernel_s:5.html
serialized_explanation/MLP#diabetes2#instance:291#num_features:8#feat_selec:none#discre:quartile#sai:

serialized_explanation/MLP#diabetes2#instance:291#num_features:8#feat_selec:forward_selection#discre:decile#sai:False#kernel_s:7.html
serialized_explanation/MLP#diabetes2#instance:291#num_features:8#feat_selec:lasso_path#discre:quartile#sai:True#kernel_s:7.html
serialized_explanation/MLP#diabetes2#instance:291#num_features:8#feat_selec:lasso_path#discre:quartile#sai:False#kernel_s:7.html
serialized_explanation/MLP#diabetes2#instance:291#num_features:8#feat_selec:lasso_path#discre:decile#sai:True#kernel_s:7.html
serialized_explanation/MLP#diabetes2#instance:291#num_features:8#feat_selec:lasso_path#discre:decile#sai:False#kernel_s:7.html
serialized_explanation/MLP#diabetes2#instance:291#num_features:8#feat_selec:none#discre:quartile#sai:True#kernel_s:7.html
serialized_explanation/MLP#diabetes2#instance:291#num_features:8#feat_selec:none#discre:quartile#sai:False#kernel_s:7.html
serialized_explanation/MLP#diabetes2#instance:291#num_features:8#feat_selec:none#discre:decile#sai:True#kernel_

## Random Forest

### Heart dataset

In [10]:
# import numpy as np
import lime
import lime.lime_tabular
import sys
sys.path.append('./ml_helper')
import ml_helper_train_model as model_train
import ml_helper_serialize_model as serializer
sys.path.append('./explanation_helper')
import expla_helper as explanation
from sklearn.model_selection import train_test_split

heart = pd.read_csv('data/datasets_33180_43520_heart.csv', sep = ',')
heart = explanation.to_float(heart)

model_file_name = 'serialized_model/RF_heart.pkl'
model = serializer.load_model(model_file_name)

dataset = heart.copy(deep = True)
target_names = np.array(['saudável', 'doente'])

folder = 'serialized_explanation/'
model_dataset = model_file_name.replace('serialized_model/','').replace('.pkl', '').replace('_', '#')


kernel_width = [math.ceil(dataset.shape[1]*0.1),
                math.ceil(dataset.shape[1]*0.3),
                math.ceil(dataset.shape[1]*0.5),
                math.ceil(dataset.shape[1]*0.7)]

for size in kernel_width:
    for sample in heart_sample:
        for f_selec in feature_selection:
            for disc in discretizer:
                for sai in sample_around_instance:
                    instance_index = sample
                    num_features = dataset.columns.size - 1 #removing the target

                    path = """{0}{1}#instance:{2}#""".format(folder, model_dataset, instance_index)
                    params_01 = """num_features:{0}#feat_selec:{1}#discre:{2}#""".format(num_features, f_selec, disc)
                    params_02 = """sai:{0}#kernel_s:{1}.html""".format(sai, size)
                    
                    final_file = """{0}{1}{2}""".format(path, params_01, params_02)
                    print(final_file)

                    explanation.explain_it_param(model,
                                               dataset,
                                               target_names,
                                               num_features,
                                               instance_index,
                                               final_file,
                                               #object level
                                               size,
                                               f_selec,     
                                               discretize_continuous,
                                               disc,
                                               sai)


serialized_explanation/RF#heart#instance:143#num_features:13#feat_selec:forward_selection#discre:quartile#sai:True#kernel_s:2.html
serialized_explanation/RF#heart#instance:143#num_features:13#feat_selec:forward_selection#discre:quartile#sai:False#kernel_s:2.html
serialized_explanation/RF#heart#instance:143#num_features:13#feat_selec:forward_selection#discre:decile#sai:True#kernel_s:2.html
serialized_explanation/RF#heart#instance:143#num_features:13#feat_selec:forward_selection#discre:decile#sai:False#kernel_s:2.html
serialized_explanation/RF#heart#instance:143#num_features:13#feat_selec:lasso_path#discre:quartile#sai:True#kernel_s:2.html
serialized_explanation/RF#heart#instance:143#num_features:13#feat_selec:lasso_path#discre:quartile#sai:False#kernel_s:2.html
serialized_explanation/RF#heart#instance:143#num_features:13#feat_selec:lasso_path#discre:decile#sai:True#kernel_s:2.html
serialized_explanation/RF#heart#instance:143#num_features:13#feat_selec:lasso_path#discre:decile#sai:False#

serialized_explanation/RF#heart#instance:143#num_features:13#feat_selec:forward_selection#discre:decile#sai:False#kernel_s:5.html
serialized_explanation/RF#heart#instance:143#num_features:13#feat_selec:lasso_path#discre:quartile#sai:True#kernel_s:5.html
serialized_explanation/RF#heart#instance:143#num_features:13#feat_selec:lasso_path#discre:quartile#sai:False#kernel_s:5.html
serialized_explanation/RF#heart#instance:143#num_features:13#feat_selec:lasso_path#discre:decile#sai:True#kernel_s:5.html
serialized_explanation/RF#heart#instance:143#num_features:13#feat_selec:lasso_path#discre:decile#sai:False#kernel_s:5.html
serialized_explanation/RF#heart#instance:143#num_features:13#feat_selec:none#discre:quartile#sai:True#kernel_s:5.html
serialized_explanation/RF#heart#instance:143#num_features:13#feat_selec:none#discre:quartile#sai:False#kernel_s:5.html
serialized_explanation/RF#heart#instance:143#num_features:13#feat_selec:none#discre:decile#sai:True#kernel_s:5.html
serialized_explanation/

serialized_explanation/RF#heart#instance:143#num_features:13#feat_selec:lasso_path#discre:decile#sai:False#kernel_s:7.html
serialized_explanation/RF#heart#instance:143#num_features:13#feat_selec:none#discre:quartile#sai:True#kernel_s:7.html
serialized_explanation/RF#heart#instance:143#num_features:13#feat_selec:none#discre:quartile#sai:False#kernel_s:7.html
serialized_explanation/RF#heart#instance:143#num_features:13#feat_selec:none#discre:decile#sai:True#kernel_s:7.html
serialized_explanation/RF#heart#instance:143#num_features:13#feat_selec:none#discre:decile#sai:False#kernel_s:7.html
serialized_explanation/RF#heart#instance:143#num_features:13#feat_selec:auto#discre:quartile#sai:True#kernel_s:7.html
serialized_explanation/RF#heart#instance:143#num_features:13#feat_selec:auto#discre:quartile#sai:False#kernel_s:7.html
serialized_explanation/RF#heart#instance:143#num_features:13#feat_selec:auto#discre:decile#sai:True#kernel_s:7.html
serialized_explanation/RF#heart#instance:143#num_featu

serialized_explanation/RF#heart#instance:143#num_features:13#feat_selec:none#discre:decile#sai:False#kernel_s:10.html
serialized_explanation/RF#heart#instance:143#num_features:13#feat_selec:auto#discre:quartile#sai:True#kernel_s:10.html
serialized_explanation/RF#heart#instance:143#num_features:13#feat_selec:auto#discre:quartile#sai:False#kernel_s:10.html
serialized_explanation/RF#heart#instance:143#num_features:13#feat_selec:auto#discre:decile#sai:True#kernel_s:10.html
serialized_explanation/RF#heart#instance:143#num_features:13#feat_selec:auto#discre:decile#sai:False#kernel_s:10.html
serialized_explanation/RF#heart#instance:203#num_features:13#feat_selec:forward_selection#discre:quartile#sai:True#kernel_s:10.html
serialized_explanation/RF#heart#instance:203#num_features:13#feat_selec:forward_selection#discre:quartile#sai:False#kernel_s:10.html
serialized_explanation/RF#heart#instance:203#num_features:13#feat_selec:forward_selection#discre:decile#sai:True#kernel_s:10.html
serialized_ex

### Wine dataset

In [11]:
# import numpy as np
import lime
import lime.lime_tabular
import sys
sys.path.append('./ml_helper')
import ml_helper_train_model as model_train
import ml_helper_serialize_model as serializer
sys.path.append('./explanation_helper')
import expla_helper as explanation
from sklearn.model_selection import train_test_split

wine = pd.read_csv('data/wine_limpo.csv', sep = ',')
# wine.drop(['Unnamed: 0'], axis = 1, inplace = True)
wine = explanation.to_float(wine)

model_file_name = 'serialized_model/RF_wine2.pkl'
model = serializer.load_model(model_file_name)

dataset = wine.copy(deep = True)
target_names = np.array(['ruim', 'bom'])

folder = 'serialized_explanation/'
model_dataset = model_file_name.replace('serialized_model/','').replace('.pkl', '').replace('_', '#')


kernel_width = [math.ceil(dataset.shape[1]*0.1),
                math.ceil(dataset.shape[1]*0.3),
                math.ceil(dataset.shape[1]*0.5),
                math.ceil(dataset.shape[1]*0.7)]

for size in kernel_width:
    for sample in wine_sample:
        for f_selec in feature_selection:
            for disc in discretizer:
                for sai in sample_around_instance:
                    instance_index = sample
                    num_features = dataset.columns.size - 1 #removing the target

                    path = """{0}{1}#instance:{2}#""".format(folder, model_dataset, instance_index)
                    params_01 = """num_features:{0}#feat_selec:{1}#discre:{2}#""".format(num_features, f_selec, disc)
                    params_02 = """sai:{0}#kernel_s:{1}.html""".format(sai, size)
                    
                    final_file = """{0}{1}{2}""".format(path, params_01, params_02)
                    print(final_file)

                    explanation.explain_it_param(model,
                                               dataset,
                                               target_names,
                                               num_features,
                                               instance_index,
                                               final_file,
                                               #object level
                                               size,
                                               f_selec,     
                                               discretize_continuous,
                                               disc,
                                               sai)


serialized_explanation/RF#wine2#instance:86#num_features:11#feat_selec:forward_selection#discre:quartile#sai:True#kernel_s:2.html
serialized_explanation/RF#wine2#instance:86#num_features:11#feat_selec:forward_selection#discre:quartile#sai:False#kernel_s:2.html
serialized_explanation/RF#wine2#instance:86#num_features:11#feat_selec:forward_selection#discre:decile#sai:True#kernel_s:2.html
serialized_explanation/RF#wine2#instance:86#num_features:11#feat_selec:forward_selection#discre:decile#sai:False#kernel_s:2.html
serialized_explanation/RF#wine2#instance:86#num_features:11#feat_selec:lasso_path#discre:quartile#sai:True#kernel_s:2.html
serialized_explanation/RF#wine2#instance:86#num_features:11#feat_selec:lasso_path#discre:quartile#sai:False#kernel_s:2.html
serialized_explanation/RF#wine2#instance:86#num_features:11#feat_selec:lasso_path#discre:decile#sai:True#kernel_s:2.html
serialized_explanation/RF#wine2#instance:86#num_features:11#feat_selec:lasso_path#discre:decile#sai:False#kernel_s

serialized_explanation/RF#wine2#instance:86#num_features:11#feat_selec:forward_selection#discre:decile#sai:False#kernel_s:4.html
serialized_explanation/RF#wine2#instance:86#num_features:11#feat_selec:lasso_path#discre:quartile#sai:True#kernel_s:4.html
serialized_explanation/RF#wine2#instance:86#num_features:11#feat_selec:lasso_path#discre:quartile#sai:False#kernel_s:4.html
serialized_explanation/RF#wine2#instance:86#num_features:11#feat_selec:lasso_path#discre:decile#sai:True#kernel_s:4.html
serialized_explanation/RF#wine2#instance:86#num_features:11#feat_selec:lasso_path#discre:decile#sai:False#kernel_s:4.html
serialized_explanation/RF#wine2#instance:86#num_features:11#feat_selec:none#discre:quartile#sai:True#kernel_s:4.html
serialized_explanation/RF#wine2#instance:86#num_features:11#feat_selec:none#discre:quartile#sai:False#kernel_s:4.html
serialized_explanation/RF#wine2#instance:86#num_features:11#feat_selec:none#discre:decile#sai:True#kernel_s:4.html
serialized_explanation/RF#wine2

serialized_explanation/RF#wine2#instance:86#num_features:11#feat_selec:lasso_path#discre:decile#sai:True#kernel_s:6.html
serialized_explanation/RF#wine2#instance:86#num_features:11#feat_selec:lasso_path#discre:decile#sai:False#kernel_s:6.html
serialized_explanation/RF#wine2#instance:86#num_features:11#feat_selec:none#discre:quartile#sai:True#kernel_s:6.html
serialized_explanation/RF#wine2#instance:86#num_features:11#feat_selec:none#discre:quartile#sai:False#kernel_s:6.html
serialized_explanation/RF#wine2#instance:86#num_features:11#feat_selec:none#discre:decile#sai:True#kernel_s:6.html
serialized_explanation/RF#wine2#instance:86#num_features:11#feat_selec:none#discre:decile#sai:False#kernel_s:6.html
serialized_explanation/RF#wine2#instance:86#num_features:11#feat_selec:auto#discre:quartile#sai:True#kernel_s:6.html
serialized_explanation/RF#wine2#instance:86#num_features:11#feat_selec:auto#discre:quartile#sai:False#kernel_s:6.html
serialized_explanation/RF#wine2#instance:86#num_features

serialized_explanation/RF#wine2#instance:86#num_features:11#feat_selec:none#discre:quartile#sai:False#kernel_s:9.html
serialized_explanation/RF#wine2#instance:86#num_features:11#feat_selec:none#discre:decile#sai:True#kernel_s:9.html
serialized_explanation/RF#wine2#instance:86#num_features:11#feat_selec:none#discre:decile#sai:False#kernel_s:9.html
serialized_explanation/RF#wine2#instance:86#num_features:11#feat_selec:auto#discre:quartile#sai:True#kernel_s:9.html
serialized_explanation/RF#wine2#instance:86#num_features:11#feat_selec:auto#discre:quartile#sai:False#kernel_s:9.html
serialized_explanation/RF#wine2#instance:86#num_features:11#feat_selec:auto#discre:decile#sai:True#kernel_s:9.html
serialized_explanation/RF#wine2#instance:86#num_features:11#feat_selec:auto#discre:decile#sai:False#kernel_s:9.html
serialized_explanation/RF#wine2#instance:1458#num_features:11#feat_selec:forward_selection#discre:quartile#sai:True#kernel_s:9.html
serialized_explanation/RF#wine2#instance:1458#num_fea

### Diabetes dataset

In [12]:
# import numpy as np
import lime
import lime.lime_tabular
import sys
sys.path.append('./ml_helper')
import ml_helper_train_model as model_train
import ml_helper_serialize_model as serializer
sys.path.append('./explanation_helper')
import expla_helper as explanation
from sklearn.model_selection import train_test_split

diabetes = pd.read_csv('data/diabetes_limpo.csv', sep = ',')
# diabetes.drop(['Unnamed: 0'], axis = 1, inplace = True)
diabetes = explanation.to_float(diabetes)

model_file_name = 'serialized_model/RF_diabetes2.pkl'
model = serializer.load_model(model_file_name)

dataset = diabetes.copy(deep = True)
target_names = np.array(['saudável', 'doente'])

folder = 'serialized_explanation/'
model_dataset = model_file_name.replace('serialized_model/','').replace('.pkl', '').replace('_', '#')


kernel_width = [math.ceil(dataset.shape[1]*0.1),
                math.ceil(dataset.shape[1]*0.3),
                math.ceil(dataset.shape[1]*0.5),
                math.ceil(dataset.shape[1]*0.7)]

for size in kernel_width:
    for sample in diabetes_sample:
        for f_selec in feature_selection:
            for disc in discretizer:
                for sai in sample_around_instance:
                    instance_index = sample
                    num_features = dataset.columns.size - 1 #removing the target

                    path = """{0}{1}#instance:{2}#""".format(folder, model_dataset, instance_index)
                    params_01 = """num_features:{0}#feat_selec:{1}#discre:{2}#""".format(num_features, f_selec, disc)
                    params_02 = """sai:{0}#kernel_s:{1}.html""".format(sai, size)
                    
                    final_file = """{0}{1}{2}""".format(path, params_01, params_02)
                    print(final_file)

                    explanation.explain_it_param(model,
                                               dataset,
                                               target_names,
                                               num_features,
                                               instance_index,
                                               final_file,
                                               #object level
                                               size,
                                               f_selec,     
                                               discretize_continuous,
                                               disc,
                                               sai)


serialized_explanation/RF#diabetes2#instance:291#num_features:8#feat_selec:forward_selection#discre:quartile#sai:True#kernel_s:1.html
serialized_explanation/RF#diabetes2#instance:291#num_features:8#feat_selec:forward_selection#discre:quartile#sai:False#kernel_s:1.html
serialized_explanation/RF#diabetes2#instance:291#num_features:8#feat_selec:forward_selection#discre:decile#sai:True#kernel_s:1.html
serialized_explanation/RF#diabetes2#instance:291#num_features:8#feat_selec:forward_selection#discre:decile#sai:False#kernel_s:1.html
serialized_explanation/RF#diabetes2#instance:291#num_features:8#feat_selec:lasso_path#discre:quartile#sai:True#kernel_s:1.html
serialized_explanation/RF#diabetes2#instance:291#num_features:8#feat_selec:lasso_path#discre:quartile#sai:False#kernel_s:1.html
serialized_explanation/RF#diabetes2#instance:291#num_features:8#feat_selec:lasso_path#discre:decile#sai:True#kernel_s:1.html
serialized_explanation/RF#diabetes2#instance:291#num_features:8#feat_selec:lasso_path#

serialized_explanation/RF#diabetes2#instance:291#num_features:8#feat_selec:forward_selection#discre:decile#sai:True#kernel_s:3.html
serialized_explanation/RF#diabetes2#instance:291#num_features:8#feat_selec:forward_selection#discre:decile#sai:False#kernel_s:3.html
serialized_explanation/RF#diabetes2#instance:291#num_features:8#feat_selec:lasso_path#discre:quartile#sai:True#kernel_s:3.html
serialized_explanation/RF#diabetes2#instance:291#num_features:8#feat_selec:lasso_path#discre:quartile#sai:False#kernel_s:3.html
serialized_explanation/RF#diabetes2#instance:291#num_features:8#feat_selec:lasso_path#discre:decile#sai:True#kernel_s:3.html
serialized_explanation/RF#diabetes2#instance:291#num_features:8#feat_selec:lasso_path#discre:decile#sai:False#kernel_s:3.html
serialized_explanation/RF#diabetes2#instance:291#num_features:8#feat_selec:none#discre:quartile#sai:True#kernel_s:3.html
serialized_explanation/RF#diabetes2#instance:291#num_features:8#feat_selec:none#discre:quartile#sai:False#ke

serialized_explanation/RF#diabetes2#instance:291#num_features:8#feat_selec:lasso_path#discre:quartile#sai:True#kernel_s:5.html
serialized_explanation/RF#diabetes2#instance:291#num_features:8#feat_selec:lasso_path#discre:quartile#sai:False#kernel_s:5.html
serialized_explanation/RF#diabetes2#instance:291#num_features:8#feat_selec:lasso_path#discre:decile#sai:True#kernel_s:5.html
serialized_explanation/RF#diabetes2#instance:291#num_features:8#feat_selec:lasso_path#discre:decile#sai:False#kernel_s:5.html
serialized_explanation/RF#diabetes2#instance:291#num_features:8#feat_selec:none#discre:quartile#sai:True#kernel_s:5.html
serialized_explanation/RF#diabetes2#instance:291#num_features:8#feat_selec:none#discre:quartile#sai:False#kernel_s:5.html
serialized_explanation/RF#diabetes2#instance:291#num_features:8#feat_selec:none#discre:decile#sai:True#kernel_s:5.html
serialized_explanation/RF#diabetes2#instance:291#num_features:8#feat_selec:none#discre:decile#sai:False#kernel_s:5.html
serialized_e

serialized_explanation/RF#diabetes2#instance:291#num_features:8#feat_selec:lasso_path#discre:decile#sai:True#kernel_s:7.html
serialized_explanation/RF#diabetes2#instance:291#num_features:8#feat_selec:lasso_path#discre:decile#sai:False#kernel_s:7.html
serialized_explanation/RF#diabetes2#instance:291#num_features:8#feat_selec:none#discre:quartile#sai:True#kernel_s:7.html
serialized_explanation/RF#diabetes2#instance:291#num_features:8#feat_selec:none#discre:quartile#sai:False#kernel_s:7.html
serialized_explanation/RF#diabetes2#instance:291#num_features:8#feat_selec:none#discre:decile#sai:True#kernel_s:7.html
serialized_explanation/RF#diabetes2#instance:291#num_features:8#feat_selec:none#discre:decile#sai:False#kernel_s:7.html
serialized_explanation/RF#diabetes2#instance:291#num_features:8#feat_selec:auto#discre:quartile#sai:True#kernel_s:7.html
serialized_explanation/RF#diabetes2#instance:291#num_features:8#feat_selec:auto#discre:quartile#sai:False#kernel_s:7.html
serialized_explanation/R

## Naive Bayes

### Heart dataset

In [13]:
# import numpy as np
import lime
import lime.lime_tabular
import sys
sys.path.append('./ml_helper')
import ml_helper_train_model as model_train
import ml_helper_serialize_model as serializer
sys.path.append('./explanation_helper')
import expla_helper as explanation
from sklearn.model_selection import train_test_split

heart = pd.read_csv('data/datasets_33180_43520_heart.csv', sep = ',')
heart = explanation.to_float(heart)

model_file_name = 'serialized_model/NaiveBayes_heart.pkl'
model = serializer.load_model(model_file_name)

dataset = heart.copy(deep = True)
target_names = np.array(['saudável', 'doente'])

folder = 'serialized_explanation/'
model_dataset = model_file_name.replace('serialized_model/','').replace('.pkl', '').replace('_', '#')


kernel_width = [math.ceil(dataset.shape[1]*0.1),
                math.ceil(dataset.shape[1]*0.3),
                math.ceil(dataset.shape[1]*0.5),
                math.ceil(dataset.shape[1]*0.7)]

for size in kernel_width:
    for sample in heart_sample:
        for f_selec in feature_selection:
            for disc in discretizer:
                for sai in sample_around_instance:
                    instance_index = sample
                    num_features = dataset.columns.size - 1 #removing the target

                    path = """{0}{1}#instance:{2}#""".format(folder, model_dataset, instance_index)
                    params_01 = """num_features:{0}#feat_selec:{1}#discre:{2}#""".format(num_features, f_selec, disc)
                    params_02 = """sai:{0}#kernel_s:{1}.html""".format(sai, size)

                    final_file = """{0}{1}{2}""".format(path, params_01, params_02)
                    print(final_file)

                    explanation.explain_it_param(model,
                                               dataset,
                                               target_names,
                                               num_features,
                                               instance_index,
                                               final_file,
                                               #object level
                                               size,
                                               f_selec,     
                                               discretize_continuous,
                                               disc,
                                               sai)


serialized_explanation/NaiveBayes#heart#instance:143#num_features:13#feat_selec:forward_selection#discre:quartile#sai:True#kernel_s:2.html
serialized_explanation/NaiveBayes#heart#instance:143#num_features:13#feat_selec:forward_selection#discre:quartile#sai:False#kernel_s:2.html
serialized_explanation/NaiveBayes#heart#instance:143#num_features:13#feat_selec:forward_selection#discre:decile#sai:True#kernel_s:2.html
serialized_explanation/NaiveBayes#heart#instance:143#num_features:13#feat_selec:forward_selection#discre:decile#sai:False#kernel_s:2.html
serialized_explanation/NaiveBayes#heart#instance:143#num_features:13#feat_selec:lasso_path#discre:quartile#sai:True#kernel_s:2.html
serialized_explanation/NaiveBayes#heart#instance:143#num_features:13#feat_selec:lasso_path#discre:quartile#sai:False#kernel_s:2.html
serialized_explanation/NaiveBayes#heart#instance:143#num_features:13#feat_selec:lasso_path#discre:decile#sai:True#kernel_s:2.html
serialized_explanation/NaiveBayes#heart#instance:14

serialized_explanation/NaiveBayes#heart#instance:96#num_features:13#feat_selec:auto#discre:decile#sai:False#kernel_s:2.html
serialized_explanation/NaiveBayes#heart#instance:143#num_features:13#feat_selec:forward_selection#discre:quartile#sai:True#kernel_s:5.html
serialized_explanation/NaiveBayes#heart#instance:143#num_features:13#feat_selec:forward_selection#discre:quartile#sai:False#kernel_s:5.html
serialized_explanation/NaiveBayes#heart#instance:143#num_features:13#feat_selec:forward_selection#discre:decile#sai:True#kernel_s:5.html
serialized_explanation/NaiveBayes#heart#instance:143#num_features:13#feat_selec:forward_selection#discre:decile#sai:False#kernel_s:5.html
serialized_explanation/NaiveBayes#heart#instance:143#num_features:13#feat_selec:lasso_path#discre:quartile#sai:True#kernel_s:5.html
serialized_explanation/NaiveBayes#heart#instance:143#num_features:13#feat_selec:lasso_path#discre:quartile#sai:False#kernel_s:5.html
serialized_explanation/NaiveBayes#heart#instance:143#num_

serialized_explanation/NaiveBayes#heart#instance:96#num_features:13#feat_selec:auto#discre:decile#sai:True#kernel_s:5.html
serialized_explanation/NaiveBayes#heart#instance:96#num_features:13#feat_selec:auto#discre:decile#sai:False#kernel_s:5.html
serialized_explanation/NaiveBayes#heart#instance:143#num_features:13#feat_selec:forward_selection#discre:quartile#sai:True#kernel_s:7.html
serialized_explanation/NaiveBayes#heart#instance:143#num_features:13#feat_selec:forward_selection#discre:quartile#sai:False#kernel_s:7.html
serialized_explanation/NaiveBayes#heart#instance:143#num_features:13#feat_selec:forward_selection#discre:decile#sai:True#kernel_s:7.html
serialized_explanation/NaiveBayes#heart#instance:143#num_features:13#feat_selec:forward_selection#discre:decile#sai:False#kernel_s:7.html
serialized_explanation/NaiveBayes#heart#instance:143#num_features:13#feat_selec:lasso_path#discre:quartile#sai:True#kernel_s:7.html
serialized_explanation/NaiveBayes#heart#instance:143#num_features:1

serialized_explanation/NaiveBayes#heart#instance:96#num_features:13#feat_selec:auto#discre:quartile#sai:False#kernel_s:7.html
serialized_explanation/NaiveBayes#heart#instance:96#num_features:13#feat_selec:auto#discre:decile#sai:True#kernel_s:7.html
serialized_explanation/NaiveBayes#heart#instance:96#num_features:13#feat_selec:auto#discre:decile#sai:False#kernel_s:7.html
serialized_explanation/NaiveBayes#heart#instance:143#num_features:13#feat_selec:forward_selection#discre:quartile#sai:True#kernel_s:10.html
serialized_explanation/NaiveBayes#heart#instance:143#num_features:13#feat_selec:forward_selection#discre:quartile#sai:False#kernel_s:10.html
serialized_explanation/NaiveBayes#heart#instance:143#num_features:13#feat_selec:forward_selection#discre:decile#sai:True#kernel_s:10.html
serialized_explanation/NaiveBayes#heart#instance:143#num_features:13#feat_selec:forward_selection#discre:decile#sai:False#kernel_s:10.html
serialized_explanation/NaiveBayes#heart#instance:143#num_features:13#

serialized_explanation/NaiveBayes#heart#instance:96#num_features:13#feat_selec:auto#discre:quartile#sai:True#kernel_s:10.html
serialized_explanation/NaiveBayes#heart#instance:96#num_features:13#feat_selec:auto#discre:quartile#sai:False#kernel_s:10.html
serialized_explanation/NaiveBayes#heart#instance:96#num_features:13#feat_selec:auto#discre:decile#sai:True#kernel_s:10.html
serialized_explanation/NaiveBayes#heart#instance:96#num_features:13#feat_selec:auto#discre:decile#sai:False#kernel_s:10.html


### Wine dataset

In [14]:
# import numpy as np
import lime
import lime.lime_tabular
import sys
sys.path.append('./ml_helper')
import ml_helper_train_model as model_train
import ml_helper_serialize_model as serializer
sys.path.append('./explanation_helper')
import expla_helper as explanation
from sklearn.model_selection import train_test_split

wine = pd.read_csv('data/wine_limpo.csv', sep = ',')
wine = explanation.to_float(wine)

model_file_name = 'serialized_model/NaiveBayes_wine2.pkl'
model = serializer.load_model(model_file_name)

dataset = wine.copy(deep = True)
target_names = np.array(['ruim', 'bom'])

folder = 'serialized_explanation/'
model_dataset = model_file_name.replace('serialized_model/','').replace('.pkl', '').replace('_', '#')

kernel_width = [math.ceil(dataset.shape[1]*0.1),
                math.ceil(dataset.shape[1]*0.3),
                math.ceil(dataset.shape[1]*0.5),
                math.ceil(dataset.shape[1]*0.7)]

for size in kernel_width:
    for sample in wine_sample:
        for f_selec in feature_selection:
            for disc in discretizer:
                for sai in sample_around_instance:
                    instance_index = sample
                    num_features = dataset.columns.size - 1 #removing the target

                    path = """{0}{1}#instance:{2}#""".format(folder, model_dataset, instance_index)
                    params_01 = """num_features:{0}#feat_selec:{1}#discre:{2}#""".format(num_features, f_selec, disc)
                    params_02 = """sai:{0}#kernel_s:{1}.html""".format(sai, size)
                    
                    final_file = """{0}{1}{2}""".format(path, params_01, params_02)
                    print(final_file)

                    explanation.explain_it_param(model,
                                               dataset,
                                               target_names,
                                               num_features,
                                               instance_index,
                                               final_file,
                                               #object level
                                               size,
                                               f_selec,     
                                               discretize_continuous,
                                               disc,
                                               sai)


serialized_explanation/NaiveBayes#wine2#instance:86#num_features:11#feat_selec:forward_selection#discre:quartile#sai:True#kernel_s:2.html
serialized_explanation/NaiveBayes#wine2#instance:86#num_features:11#feat_selec:forward_selection#discre:quartile#sai:False#kernel_s:2.html
serialized_explanation/NaiveBayes#wine2#instance:86#num_features:11#feat_selec:forward_selection#discre:decile#sai:True#kernel_s:2.html
serialized_explanation/NaiveBayes#wine2#instance:86#num_features:11#feat_selec:forward_selection#discre:decile#sai:False#kernel_s:2.html
serialized_explanation/NaiveBayes#wine2#instance:86#num_features:11#feat_selec:lasso_path#discre:quartile#sai:True#kernel_s:2.html
serialized_explanation/NaiveBayes#wine2#instance:86#num_features:11#feat_selec:lasso_path#discre:quartile#sai:False#kernel_s:2.html
serialized_explanation/NaiveBayes#wine2#instance:86#num_features:11#feat_selec:lasso_path#discre:decile#sai:True#kernel_s:2.html
serialized_explanation/NaiveBayes#wine2#instance:86#num_fe

serialized_explanation/NaiveBayes#wine2#instance:1003#num_features:11#feat_selec:auto#discre:decile#sai:False#kernel_s:2.html
serialized_explanation/NaiveBayes#wine2#instance:86#num_features:11#feat_selec:forward_selection#discre:quartile#sai:True#kernel_s:4.html
serialized_explanation/NaiveBayes#wine2#instance:86#num_features:11#feat_selec:forward_selection#discre:quartile#sai:False#kernel_s:4.html
serialized_explanation/NaiveBayes#wine2#instance:86#num_features:11#feat_selec:forward_selection#discre:decile#sai:True#kernel_s:4.html
serialized_explanation/NaiveBayes#wine2#instance:86#num_features:11#feat_selec:forward_selection#discre:decile#sai:False#kernel_s:4.html
serialized_explanation/NaiveBayes#wine2#instance:86#num_features:11#feat_selec:lasso_path#discre:quartile#sai:True#kernel_s:4.html
serialized_explanation/NaiveBayes#wine2#instance:86#num_features:11#feat_selec:lasso_path#discre:quartile#sai:False#kernel_s:4.html
serialized_explanation/NaiveBayes#wine2#instance:86#num_featu

serialized_explanation/NaiveBayes#wine2#instance:1003#num_features:11#feat_selec:auto#discre:decile#sai:True#kernel_s:4.html
serialized_explanation/NaiveBayes#wine2#instance:1003#num_features:11#feat_selec:auto#discre:decile#sai:False#kernel_s:4.html
serialized_explanation/NaiveBayes#wine2#instance:86#num_features:11#feat_selec:forward_selection#discre:quartile#sai:True#kernel_s:6.html
serialized_explanation/NaiveBayes#wine2#instance:86#num_features:11#feat_selec:forward_selection#discre:quartile#sai:False#kernel_s:6.html
serialized_explanation/NaiveBayes#wine2#instance:86#num_features:11#feat_selec:forward_selection#discre:decile#sai:True#kernel_s:6.html
serialized_explanation/NaiveBayes#wine2#instance:86#num_features:11#feat_selec:forward_selection#discre:decile#sai:False#kernel_s:6.html
serialized_explanation/NaiveBayes#wine2#instance:86#num_features:11#feat_selec:lasso_path#discre:quartile#sai:True#kernel_s:6.html
serialized_explanation/NaiveBayes#wine2#instance:86#num_features:11#

serialized_explanation/NaiveBayes#wine2#instance:1003#num_features:11#feat_selec:auto#discre:quartile#sai:False#kernel_s:6.html
serialized_explanation/NaiveBayes#wine2#instance:1003#num_features:11#feat_selec:auto#discre:decile#sai:True#kernel_s:6.html
serialized_explanation/NaiveBayes#wine2#instance:1003#num_features:11#feat_selec:auto#discre:decile#sai:False#kernel_s:6.html
serialized_explanation/NaiveBayes#wine2#instance:86#num_features:11#feat_selec:forward_selection#discre:quartile#sai:True#kernel_s:9.html
serialized_explanation/NaiveBayes#wine2#instance:86#num_features:11#feat_selec:forward_selection#discre:quartile#sai:False#kernel_s:9.html
serialized_explanation/NaiveBayes#wine2#instance:86#num_features:11#feat_selec:forward_selection#discre:decile#sai:True#kernel_s:9.html
serialized_explanation/NaiveBayes#wine2#instance:86#num_features:11#feat_selec:forward_selection#discre:decile#sai:False#kernel_s:9.html
serialized_explanation/NaiveBayes#wine2#instance:86#num_features:11#fea

serialized_explanation/NaiveBayes#wine2#instance:1003#num_features:11#feat_selec:auto#discre:quartile#sai:True#kernel_s:9.html
serialized_explanation/NaiveBayes#wine2#instance:1003#num_features:11#feat_selec:auto#discre:quartile#sai:False#kernel_s:9.html
serialized_explanation/NaiveBayes#wine2#instance:1003#num_features:11#feat_selec:auto#discre:decile#sai:True#kernel_s:9.html
serialized_explanation/NaiveBayes#wine2#instance:1003#num_features:11#feat_selec:auto#discre:decile#sai:False#kernel_s:9.html


### Diabetes dataset

In [15]:
# import numpy as np
import lime
import lime.lime_tabular
import sys
sys.path.append('./ml_helper')
import ml_helper_train_model as model_train
import ml_helper_serialize_model as serializer
sys.path.append('./explanation_helper')
import expla_helper as explanation
from sklearn.model_selection import train_test_split

diabetes = pd.read_csv('data/diabetes_limpo.csv', sep = ',')
# diabetes.drop(['Unnamed: 0'], axis = 1, inplace = True)
diabetes = explanation.to_float(diabetes)

model_file_name = 'serialized_model/NaiveBayes_diabetes2.pkl'
model = serializer.load_model(model_file_name)

dataset = diabetes.copy(deep = True)
target_names = np.array(['saudável', 'doente'])

folder = 'serialized_explanation/'
model_dataset = model_file_name.replace('serialized_model/','').replace('.pkl', '').replace('_', '#')

kernel_width = [math.ceil(dataset.shape[1]*0.1),
                math.ceil(dataset.shape[1]*0.3),
                math.ceil(dataset.shape[1]*0.5),
                math.ceil(dataset.shape[1]*0.7)]

for size in kernel_width:
    for sample in diabetes_sample:
        for f_selec in feature_selection:
            for disc in discretizer:
                for sai in sample_around_instance:
                    instance_index = sample
                    num_features = dataset.columns.size - 1 #removing the target

                    path = """{0}{1}#instance:{2}#""".format(folder, model_dataset, instance_index)
                    params_01 = """num_features:{0}#feat_selec:{1}#discre:{2}#""".format(num_features, f_selec, disc)
                    params_02 = """sai:{0}#kernel_s:{1}.html""".format(sai, size)
                    
                    final_file = """{0}{1}{2}""".format(path, params_01, params_02)
                    print(final_file)

                    explanation.explain_it_param(model,
                                               dataset,
                                               target_names,
                                               num_features,
                                               instance_index,
                                               final_file,
                                               #object level
                                               size,
                                               f_selec,     
                                               discretize_continuous,
                                               disc,
                                               sai)


serialized_explanation/NaiveBayes#diabetes2#instance:291#num_features:8#feat_selec:forward_selection#discre:quartile#sai:True#kernel_s:1.html
serialized_explanation/NaiveBayes#diabetes2#instance:291#num_features:8#feat_selec:forward_selection#discre:quartile#sai:False#kernel_s:1.html
serialized_explanation/NaiveBayes#diabetes2#instance:291#num_features:8#feat_selec:forward_selection#discre:decile#sai:True#kernel_s:1.html
serialized_explanation/NaiveBayes#diabetes2#instance:291#num_features:8#feat_selec:forward_selection#discre:decile#sai:False#kernel_s:1.html
serialized_explanation/NaiveBayes#diabetes2#instance:291#num_features:8#feat_selec:lasso_path#discre:quartile#sai:True#kernel_s:1.html
serialized_explanation/NaiveBayes#diabetes2#instance:291#num_features:8#feat_selec:lasso_path#discre:quartile#sai:False#kernel_s:1.html
serialized_explanation/NaiveBayes#diabetes2#instance:291#num_features:8#feat_selec:lasso_path#discre:decile#sai:True#kernel_s:1.html
serialized_explanation/NaiveBa

serialized_explanation/NaiveBayes#diabetes2#instance:755#num_features:8#feat_selec:auto#discre:decile#sai:True#kernel_s:1.html
serialized_explanation/NaiveBayes#diabetes2#instance:755#num_features:8#feat_selec:auto#discre:decile#sai:False#kernel_s:1.html
serialized_explanation/NaiveBayes#diabetes2#instance:291#num_features:8#feat_selec:forward_selection#discre:quartile#sai:True#kernel_s:3.html
serialized_explanation/NaiveBayes#diabetes2#instance:291#num_features:8#feat_selec:forward_selection#discre:quartile#sai:False#kernel_s:3.html
serialized_explanation/NaiveBayes#diabetes2#instance:291#num_features:8#feat_selec:forward_selection#discre:decile#sai:True#kernel_s:3.html
serialized_explanation/NaiveBayes#diabetes2#instance:291#num_features:8#feat_selec:forward_selection#discre:decile#sai:False#kernel_s:3.html
serialized_explanation/NaiveBayes#diabetes2#instance:291#num_features:8#feat_selec:lasso_path#discre:quartile#sai:True#kernel_s:3.html
serialized_explanation/NaiveBayes#diabetes2#

serialized_explanation/NaiveBayes#diabetes2#instance:755#num_features:8#feat_selec:auto#discre:quartile#sai:True#kernel_s:3.html
serialized_explanation/NaiveBayes#diabetes2#instance:755#num_features:8#feat_selec:auto#discre:quartile#sai:False#kernel_s:3.html
serialized_explanation/NaiveBayes#diabetes2#instance:755#num_features:8#feat_selec:auto#discre:decile#sai:True#kernel_s:3.html
serialized_explanation/NaiveBayes#diabetes2#instance:755#num_features:8#feat_selec:auto#discre:decile#sai:False#kernel_s:3.html
serialized_explanation/NaiveBayes#diabetes2#instance:291#num_features:8#feat_selec:forward_selection#discre:quartile#sai:True#kernel_s:5.html
serialized_explanation/NaiveBayes#diabetes2#instance:291#num_features:8#feat_selec:forward_selection#discre:quartile#sai:False#kernel_s:5.html
serialized_explanation/NaiveBayes#diabetes2#instance:291#num_features:8#feat_selec:forward_selection#discre:decile#sai:True#kernel_s:5.html
serialized_explanation/NaiveBayes#diabetes2#instance:291#num_

serialized_explanation/NaiveBayes#diabetes2#instance:755#num_features:8#feat_selec:none#discre:decile#sai:True#kernel_s:5.html
serialized_explanation/NaiveBayes#diabetes2#instance:755#num_features:8#feat_selec:none#discre:decile#sai:False#kernel_s:5.html
serialized_explanation/NaiveBayes#diabetes2#instance:755#num_features:8#feat_selec:auto#discre:quartile#sai:True#kernel_s:5.html
serialized_explanation/NaiveBayes#diabetes2#instance:755#num_features:8#feat_selec:auto#discre:quartile#sai:False#kernel_s:5.html
serialized_explanation/NaiveBayes#diabetes2#instance:755#num_features:8#feat_selec:auto#discre:decile#sai:True#kernel_s:5.html
serialized_explanation/NaiveBayes#diabetes2#instance:755#num_features:8#feat_selec:auto#discre:decile#sai:False#kernel_s:5.html
serialized_explanation/NaiveBayes#diabetes2#instance:291#num_features:8#feat_selec:forward_selection#discre:quartile#sai:True#kernel_s:7.html
serialized_explanation/NaiveBayes#diabetes2#instance:291#num_features:8#feat_selec:forwar

serialized_explanation/NaiveBayes#diabetes2#instance:755#num_features:8#feat_selec:none#discre:quartile#sai:True#kernel_s:7.html
serialized_explanation/NaiveBayes#diabetes2#instance:755#num_features:8#feat_selec:none#discre:quartile#sai:False#kernel_s:7.html
serialized_explanation/NaiveBayes#diabetes2#instance:755#num_features:8#feat_selec:none#discre:decile#sai:True#kernel_s:7.html
serialized_explanation/NaiveBayes#diabetes2#instance:755#num_features:8#feat_selec:none#discre:decile#sai:False#kernel_s:7.html
serialized_explanation/NaiveBayes#diabetes2#instance:755#num_features:8#feat_selec:auto#discre:quartile#sai:True#kernel_s:7.html
serialized_explanation/NaiveBayes#diabetes2#instance:755#num_features:8#feat_selec:auto#discre:quartile#sai:False#kernel_s:7.html
serialized_explanation/NaiveBayes#diabetes2#instance:755#num_features:8#feat_selec:auto#discre:decile#sai:True#kernel_s:7.html
serialized_explanation/NaiveBayes#diabetes2#instance:755#num_features:8#feat_selec:auto#discre:decil

## Decision Tree

### Heart dataset

In [16]:
# import numpy as np
import lime
import lime.lime_tabular
import sys
sys.path.append('./ml_helper')
import ml_helper_train_model as model_train
import ml_helper_serialize_model as serializer
sys.path.append('./explanation_helper')
import expla_helper as explanation
from sklearn.model_selection import train_test_split

heart = pd.read_csv('data/datasets_33180_43520_heart.csv', sep = ',')
heart = explanation.to_float(heart)

model_file_name = 'serialized_model/DT_heart.pkl'
model = serializer.load_model(model_file_name)

dataset = heart.copy(deep = True)
target_names = np.array(['saudável', 'doente'])

folder = 'serialized_explanation/'
model_dataset = model_file_name.replace('serialized_model/','').replace('.pkl', '').replace('_', '#')

kernel_width = [math.ceil(dataset.shape[1]*0.1),
                math.ceil(dataset.shape[1]*0.3),
                math.ceil(dataset.shape[1]*0.5),
                math.ceil(dataset.shape[1]*0.7)]

for size in kernel_width:
    for sample in heart_sample:
        for f_selec in feature_selection:
            for disc in discretizer:
                for sai in sample_around_instance:
                    instance_index = sample
                    num_features = dataset.columns.size - 1 #removing the target

                    path = """{0}{1}#instance:{2}#""".format(folder, model_dataset, instance_index)
                    params_01 = """num_features:{0}#feat_selec:{1}#discre:{2}#""".format(num_features, f_selec, disc)
                    params_02 = """sai:{0}#kernel_s:{1}.html""".format(sai, size)
                    
                    final_file = """{0}{1}{2}""".format(path, params_01, params_02)
                    print(final_file)

                    explanation.explain_it_param(model,
                                               dataset,
                                               target_names,
                                               num_features,
                                               instance_index,
                                               final_file,
                                               #object level
                                               size,
                                               f_selec,     
                                               discretize_continuous,
                                               disc,
                                               sai)


serialized_explanation/DT#heart#instance:143#num_features:13#feat_selec:forward_selection#discre:quartile#sai:True#kernel_s:2.html
serialized_explanation/DT#heart#instance:143#num_features:13#feat_selec:forward_selection#discre:quartile#sai:False#kernel_s:2.html
serialized_explanation/DT#heart#instance:143#num_features:13#feat_selec:forward_selection#discre:decile#sai:True#kernel_s:2.html
serialized_explanation/DT#heart#instance:143#num_features:13#feat_selec:forward_selection#discre:decile#sai:False#kernel_s:2.html
serialized_explanation/DT#heart#instance:143#num_features:13#feat_selec:lasso_path#discre:quartile#sai:True#kernel_s:2.html
serialized_explanation/DT#heart#instance:143#num_features:13#feat_selec:lasso_path#discre:quartile#sai:False#kernel_s:2.html
serialized_explanation/DT#heart#instance:143#num_features:13#feat_selec:lasso_path#discre:decile#sai:True#kernel_s:2.html
serialized_explanation/DT#heart#instance:143#num_features:13#feat_selec:lasso_path#discre:decile#sai:False#

serialized_explanation/DT#heart#instance:143#num_features:13#feat_selec:forward_selection#discre:decile#sai:False#kernel_s:5.html
serialized_explanation/DT#heart#instance:143#num_features:13#feat_selec:lasso_path#discre:quartile#sai:True#kernel_s:5.html
serialized_explanation/DT#heart#instance:143#num_features:13#feat_selec:lasso_path#discre:quartile#sai:False#kernel_s:5.html
serialized_explanation/DT#heart#instance:143#num_features:13#feat_selec:lasso_path#discre:decile#sai:True#kernel_s:5.html
serialized_explanation/DT#heart#instance:143#num_features:13#feat_selec:lasso_path#discre:decile#sai:False#kernel_s:5.html
serialized_explanation/DT#heart#instance:143#num_features:13#feat_selec:none#discre:quartile#sai:True#kernel_s:5.html
serialized_explanation/DT#heart#instance:143#num_features:13#feat_selec:none#discre:quartile#sai:False#kernel_s:5.html
serialized_explanation/DT#heart#instance:143#num_features:13#feat_selec:none#discre:decile#sai:True#kernel_s:5.html
serialized_explanation/

serialized_explanation/DT#heart#instance:143#num_features:13#feat_selec:lasso_path#discre:decile#sai:False#kernel_s:7.html
serialized_explanation/DT#heart#instance:143#num_features:13#feat_selec:none#discre:quartile#sai:True#kernel_s:7.html
serialized_explanation/DT#heart#instance:143#num_features:13#feat_selec:none#discre:quartile#sai:False#kernel_s:7.html
serialized_explanation/DT#heart#instance:143#num_features:13#feat_selec:none#discre:decile#sai:True#kernel_s:7.html
serialized_explanation/DT#heart#instance:143#num_features:13#feat_selec:none#discre:decile#sai:False#kernel_s:7.html
serialized_explanation/DT#heart#instance:143#num_features:13#feat_selec:auto#discre:quartile#sai:True#kernel_s:7.html
serialized_explanation/DT#heart#instance:143#num_features:13#feat_selec:auto#discre:quartile#sai:False#kernel_s:7.html
serialized_explanation/DT#heart#instance:143#num_features:13#feat_selec:auto#discre:decile#sai:True#kernel_s:7.html
serialized_explanation/DT#heart#instance:143#num_featu

serialized_explanation/DT#heart#instance:143#num_features:13#feat_selec:none#discre:decile#sai:False#kernel_s:10.html
serialized_explanation/DT#heart#instance:143#num_features:13#feat_selec:auto#discre:quartile#sai:True#kernel_s:10.html
serialized_explanation/DT#heart#instance:143#num_features:13#feat_selec:auto#discre:quartile#sai:False#kernel_s:10.html
serialized_explanation/DT#heart#instance:143#num_features:13#feat_selec:auto#discre:decile#sai:True#kernel_s:10.html
serialized_explanation/DT#heart#instance:143#num_features:13#feat_selec:auto#discre:decile#sai:False#kernel_s:10.html
serialized_explanation/DT#heart#instance:203#num_features:13#feat_selec:forward_selection#discre:quartile#sai:True#kernel_s:10.html
serialized_explanation/DT#heart#instance:203#num_features:13#feat_selec:forward_selection#discre:quartile#sai:False#kernel_s:10.html
serialized_explanation/DT#heart#instance:203#num_features:13#feat_selec:forward_selection#discre:decile#sai:True#kernel_s:10.html
serialized_ex

### Wine dataset

In [17]:
# import numpy as np
import lime
import lime.lime_tabular
import sys
sys.path.append('./ml_helper')
import ml_helper_train_model as model_train
import ml_helper_serialize_model as serializer
sys.path.append('./explanation_helper')
import expla_helper as explanation
from sklearn.model_selection import train_test_split

wine = pd.read_csv('data/wine_limpo.csv', sep = ',')
# wine.drop(['Unnamed: 0'], axis = 1, inplace = True)
wine = explanation.to_float(wine)

model_file_name = 'serialized_model/DT_wine2.pkl'
model = serializer.load_model(model_file_name)

dataset = wine.copy(deep = True)
target_names = np.array(['ruim', 'bom'])

folder = 'serialized_explanation/'
model_dataset = model_file_name.replace('serialized_model/','').replace('.pkl', '').replace('_', '#')

kernel_width = [math.ceil(dataset.shape[1]*0.1),
                math.ceil(dataset.shape[1]*0.3),
                math.ceil(dataset.shape[1]*0.5),
                math.ceil(dataset.shape[1]*0.7)]

for size in kernel_width:
    for sample in wine_sample:
        for f_selec in feature_selection:
            for disc in discretizer:
                for sai in sample_around_instance:
                    instance_index = sample
                    num_features = dataset.columns.size - 1 #removing the target

                    path = """{0}{1}#instance:{2}#""".format(folder, model_dataset, instance_index)
                    params_01 = """num_features:{0}#feat_selec:{1}#discre:{2}#""".format(num_features, f_selec, disc)
                    params_02 = """sai:{0}#kernel_s:{1}.html""".format(sai, size)
                    
                    final_file = """{0}{1}{2}""".format(path, params_01, params_02)
                    print(final_file)

                    explanation.explain_it_param(model,
                                               dataset,
                                               target_names,
                                               num_features,
                                               instance_index,
                                               final_file,
                                               #object level
                                               size,
                                               f_selec,     
                                               discretize_continuous,
                                               disc,
                                               sai)


serialized_explanation/DT#wine2#instance:86#num_features:11#feat_selec:forward_selection#discre:quartile#sai:True#kernel_s:2.html
serialized_explanation/DT#wine2#instance:86#num_features:11#feat_selec:forward_selection#discre:quartile#sai:False#kernel_s:2.html
serialized_explanation/DT#wine2#instance:86#num_features:11#feat_selec:forward_selection#discre:decile#sai:True#kernel_s:2.html
serialized_explanation/DT#wine2#instance:86#num_features:11#feat_selec:forward_selection#discre:decile#sai:False#kernel_s:2.html
serialized_explanation/DT#wine2#instance:86#num_features:11#feat_selec:lasso_path#discre:quartile#sai:True#kernel_s:2.html
serialized_explanation/DT#wine2#instance:86#num_features:11#feat_selec:lasso_path#discre:quartile#sai:False#kernel_s:2.html
serialized_explanation/DT#wine2#instance:86#num_features:11#feat_selec:lasso_path#discre:decile#sai:True#kernel_s:2.html
serialized_explanation/DT#wine2#instance:86#num_features:11#feat_selec:lasso_path#discre:decile#sai:False#kernel_s

serialized_explanation/DT#wine2#instance:86#num_features:11#feat_selec:forward_selection#discre:decile#sai:False#kernel_s:4.html
serialized_explanation/DT#wine2#instance:86#num_features:11#feat_selec:lasso_path#discre:quartile#sai:True#kernel_s:4.html
serialized_explanation/DT#wine2#instance:86#num_features:11#feat_selec:lasso_path#discre:quartile#sai:False#kernel_s:4.html
serialized_explanation/DT#wine2#instance:86#num_features:11#feat_selec:lasso_path#discre:decile#sai:True#kernel_s:4.html
serialized_explanation/DT#wine2#instance:86#num_features:11#feat_selec:lasso_path#discre:decile#sai:False#kernel_s:4.html
serialized_explanation/DT#wine2#instance:86#num_features:11#feat_selec:none#discre:quartile#sai:True#kernel_s:4.html
serialized_explanation/DT#wine2#instance:86#num_features:11#feat_selec:none#discre:quartile#sai:False#kernel_s:4.html
serialized_explanation/DT#wine2#instance:86#num_features:11#feat_selec:none#discre:decile#sai:True#kernel_s:4.html
serialized_explanation/DT#wine2

serialized_explanation/DT#wine2#instance:86#num_features:11#feat_selec:lasso_path#discre:decile#sai:True#kernel_s:6.html
serialized_explanation/DT#wine2#instance:86#num_features:11#feat_selec:lasso_path#discre:decile#sai:False#kernel_s:6.html
serialized_explanation/DT#wine2#instance:86#num_features:11#feat_selec:none#discre:quartile#sai:True#kernel_s:6.html
serialized_explanation/DT#wine2#instance:86#num_features:11#feat_selec:none#discre:quartile#sai:False#kernel_s:6.html
serialized_explanation/DT#wine2#instance:86#num_features:11#feat_selec:none#discre:decile#sai:True#kernel_s:6.html
serialized_explanation/DT#wine2#instance:86#num_features:11#feat_selec:none#discre:decile#sai:False#kernel_s:6.html
serialized_explanation/DT#wine2#instance:86#num_features:11#feat_selec:auto#discre:quartile#sai:True#kernel_s:6.html
serialized_explanation/DT#wine2#instance:86#num_features:11#feat_selec:auto#discre:quartile#sai:False#kernel_s:6.html
serialized_explanation/DT#wine2#instance:86#num_features

serialized_explanation/DT#wine2#instance:86#num_features:11#feat_selec:none#discre:quartile#sai:False#kernel_s:9.html
serialized_explanation/DT#wine2#instance:86#num_features:11#feat_selec:none#discre:decile#sai:True#kernel_s:9.html
serialized_explanation/DT#wine2#instance:86#num_features:11#feat_selec:none#discre:decile#sai:False#kernel_s:9.html
serialized_explanation/DT#wine2#instance:86#num_features:11#feat_selec:auto#discre:quartile#sai:True#kernel_s:9.html
serialized_explanation/DT#wine2#instance:86#num_features:11#feat_selec:auto#discre:quartile#sai:False#kernel_s:9.html
serialized_explanation/DT#wine2#instance:86#num_features:11#feat_selec:auto#discre:decile#sai:True#kernel_s:9.html
serialized_explanation/DT#wine2#instance:86#num_features:11#feat_selec:auto#discre:decile#sai:False#kernel_s:9.html
serialized_explanation/DT#wine2#instance:1458#num_features:11#feat_selec:forward_selection#discre:quartile#sai:True#kernel_s:9.html
serialized_explanation/DT#wine2#instance:1458#num_fea

### Diabetes dataset

In [18]:
# import numpy as np
import lime
import lime.lime_tabular
import sys
sys.path.append('./ml_helper')
import ml_helper_train_model as model_train
import ml_helper_serialize_model as serializer
sys.path.append('./explanation_helper')
import expla_helper as explanation
from sklearn.model_selection import train_test_split

diabetes = pd.read_csv('data/diabetes_limpo.csv', sep = ',')
# diabetes.drop(['Unnamed: 0'], axis = 1, inplace = True)
diabetes = explanation.to_float(diabetes)

model_file_name = 'serialized_model/DT_diabetes2.pkl'
model = serializer.load_model(model_file_name)

dataset = diabetes.copy(deep = True)
target_names = np.array(['saudável', 'doente'])

folder = 'serialized_explanation/'
model_dataset = model_file_name.replace('serialized_model/','').replace('.pkl', '').replace('_', '#')


kernel_width = [math.ceil(dataset.shape[1]*0.1),
                math.ceil(dataset.shape[1]*0.3),
                math.ceil(dataset.shape[1]*0.5),
                math.ceil(dataset.shape[1]*0.7)]

for size in kernel_width:
    for sample in diabetes_sample:
        for f_selec in feature_selection:
            for disc in discretizer:
                for sai in sample_around_instance:
                    instance_index = sample
                    num_features = dataset.columns.size - 1 #removing the target

                    path = """{0}{1}#instance:{2}#""".format(folder, model_dataset, instance_index)
                    params_01 = """num_features:{0}#feat_selec:{1}#discre:{2}#""".format(num_features, f_selec, disc)
                    params_02 = """sai:{0}#kernel_s:{1}.html""".format(sai, size)
                    
                    final_file = """{0}{1}{2}""".format(path, params_01, params_02)
                    print(final_file)

                    explanation.explain_it_param(model,
                                               dataset,
                                               target_names,
                                               num_features,
                                               instance_index,
                                               final_file,
                                               #object level
                                               size,
                                               f_selec,     
                                               discretize_continuous,
                                               disc,
                                               sai)


serialized_explanation/DT#diabetes2#instance:291#num_features:8#feat_selec:forward_selection#discre:quartile#sai:True#kernel_s:1.html
serialized_explanation/DT#diabetes2#instance:291#num_features:8#feat_selec:forward_selection#discre:quartile#sai:False#kernel_s:1.html
serialized_explanation/DT#diabetes2#instance:291#num_features:8#feat_selec:forward_selection#discre:decile#sai:True#kernel_s:1.html
serialized_explanation/DT#diabetes2#instance:291#num_features:8#feat_selec:forward_selection#discre:decile#sai:False#kernel_s:1.html
serialized_explanation/DT#diabetes2#instance:291#num_features:8#feat_selec:lasso_path#discre:quartile#sai:True#kernel_s:1.html
serialized_explanation/DT#diabetes2#instance:291#num_features:8#feat_selec:lasso_path#discre:quartile#sai:False#kernel_s:1.html
serialized_explanation/DT#diabetes2#instance:291#num_features:8#feat_selec:lasso_path#discre:decile#sai:True#kernel_s:1.html
serialized_explanation/DT#diabetes2#instance:291#num_features:8#feat_selec:lasso_path#

serialized_explanation/DT#diabetes2#instance:291#num_features:8#feat_selec:forward_selection#discre:decile#sai:True#kernel_s:3.html
serialized_explanation/DT#diabetes2#instance:291#num_features:8#feat_selec:forward_selection#discre:decile#sai:False#kernel_s:3.html
serialized_explanation/DT#diabetes2#instance:291#num_features:8#feat_selec:lasso_path#discre:quartile#sai:True#kernel_s:3.html
serialized_explanation/DT#diabetes2#instance:291#num_features:8#feat_selec:lasso_path#discre:quartile#sai:False#kernel_s:3.html
serialized_explanation/DT#diabetes2#instance:291#num_features:8#feat_selec:lasso_path#discre:decile#sai:True#kernel_s:3.html
serialized_explanation/DT#diabetes2#instance:291#num_features:8#feat_selec:lasso_path#discre:decile#sai:False#kernel_s:3.html
serialized_explanation/DT#diabetes2#instance:291#num_features:8#feat_selec:none#discre:quartile#sai:True#kernel_s:3.html
serialized_explanation/DT#diabetes2#instance:291#num_features:8#feat_selec:none#discre:quartile#sai:False#ke

serialized_explanation/DT#diabetes2#instance:291#num_features:8#feat_selec:lasso_path#discre:quartile#sai:True#kernel_s:5.html
serialized_explanation/DT#diabetes2#instance:291#num_features:8#feat_selec:lasso_path#discre:quartile#sai:False#kernel_s:5.html
serialized_explanation/DT#diabetes2#instance:291#num_features:8#feat_selec:lasso_path#discre:decile#sai:True#kernel_s:5.html
serialized_explanation/DT#diabetes2#instance:291#num_features:8#feat_selec:lasso_path#discre:decile#sai:False#kernel_s:5.html
serialized_explanation/DT#diabetes2#instance:291#num_features:8#feat_selec:none#discre:quartile#sai:True#kernel_s:5.html
serialized_explanation/DT#diabetes2#instance:291#num_features:8#feat_selec:none#discre:quartile#sai:False#kernel_s:5.html
serialized_explanation/DT#diabetes2#instance:291#num_features:8#feat_selec:none#discre:decile#sai:True#kernel_s:5.html
serialized_explanation/DT#diabetes2#instance:291#num_features:8#feat_selec:none#discre:decile#sai:False#kernel_s:5.html
serialized_e

serialized_explanation/DT#diabetes2#instance:291#num_features:8#feat_selec:lasso_path#discre:decile#sai:True#kernel_s:7.html
serialized_explanation/DT#diabetes2#instance:291#num_features:8#feat_selec:lasso_path#discre:decile#sai:False#kernel_s:7.html
serialized_explanation/DT#diabetes2#instance:291#num_features:8#feat_selec:none#discre:quartile#sai:True#kernel_s:7.html
serialized_explanation/DT#diabetes2#instance:291#num_features:8#feat_selec:none#discre:quartile#sai:False#kernel_s:7.html
serialized_explanation/DT#diabetes2#instance:291#num_features:8#feat_selec:none#discre:decile#sai:True#kernel_s:7.html
serialized_explanation/DT#diabetes2#instance:291#num_features:8#feat_selec:none#discre:decile#sai:False#kernel_s:7.html
serialized_explanation/DT#diabetes2#instance:291#num_features:8#feat_selec:auto#discre:quartile#sai:True#kernel_s:7.html
serialized_explanation/DT#diabetes2#instance:291#num_features:8#feat_selec:auto#discre:quartile#sai:False#kernel_s:7.html
serialized_explanation/D

## KNN

### Heart dataset

In [19]:
# import numpy as np
import lime
import lime.lime_tabular
import sys
sys.path.append('./ml_helper')
import ml_helper_train_model as model_train
import ml_helper_serialize_model as serializer
sys.path.append('./explanation_helper')
import expla_helper as explanation
from sklearn.model_selection import train_test_split

heart = pd.read_csv('data/datasets_33180_43520_heart.csv', sep = ',')
heart = explanation.to_float(heart)

model_file_name = 'serialized_model/KNN_heart.pkl'
model = serializer.load_model(model_file_name)

dataset = heart.copy(deep = True)
target_names = np.array(['saudável', 'doente'])

folder = 'serialized_explanation/'
model_dataset = model_file_name.replace('serialized_model/','').replace('.pkl', '').replace('_', '#')


kernel_width = [math.ceil(dataset.shape[1]*0.1),
                math.ceil(dataset.shape[1]*0.3),
                math.ceil(dataset.shape[1]*0.5),
                math.ceil(dataset.shape[1]*0.7)]

for size in kernel_width:
    for sample in heart_sample:
        for f_selec in feature_selection:
            for disc in discretizer:
                for sai in sample_around_instance:
                    instance_index = sample
                    num_features = dataset.columns.size - 1 #removing the target

                    path = """{0}{1}#instance:{2}#""".format(folder, model_dataset, instance_index)
                    params_01 = """num_features:{0}#feat_selec:{1}#discre:{2}#""".format(num_features, f_selec, disc)
                    params_02 = """sai:{0}#kernel_s:{1}.html""".format(sai, size)
                    
                    final_file = """{0}{1}{2}""".format(path, params_01, params_02)
                    print(final_file)

                    explanation.explain_it_param(model,
                                               dataset,
                                               target_names,
                                               num_features,
                                               instance_index,
                                               final_file,
                                               #object level
                                               size,
                                               f_selec,     
                                               discretize_continuous,
                                               disc,
                                               sai)


serialized_explanation/KNN#heart#instance:143#num_features:13#feat_selec:forward_selection#discre:quartile#sai:True#kernel_s:2.html
serialized_explanation/KNN#heart#instance:143#num_features:13#feat_selec:forward_selection#discre:quartile#sai:False#kernel_s:2.html
serialized_explanation/KNN#heart#instance:143#num_features:13#feat_selec:forward_selection#discre:decile#sai:True#kernel_s:2.html
serialized_explanation/KNN#heart#instance:143#num_features:13#feat_selec:forward_selection#discre:decile#sai:False#kernel_s:2.html
serialized_explanation/KNN#heart#instance:143#num_features:13#feat_selec:lasso_path#discre:quartile#sai:True#kernel_s:2.html
serialized_explanation/KNN#heart#instance:143#num_features:13#feat_selec:lasso_path#discre:quartile#sai:False#kernel_s:2.html
serialized_explanation/KNN#heart#instance:143#num_features:13#feat_selec:lasso_path#discre:decile#sai:True#kernel_s:2.html
serialized_explanation/KNN#heart#instance:143#num_features:13#feat_selec:lasso_path#discre:decile#sa

serialized_explanation/KNN#heart#instance:143#num_features:13#feat_selec:forward_selection#discre:decile#sai:False#kernel_s:5.html
serialized_explanation/KNN#heart#instance:143#num_features:13#feat_selec:lasso_path#discre:quartile#sai:True#kernel_s:5.html
serialized_explanation/KNN#heart#instance:143#num_features:13#feat_selec:lasso_path#discre:quartile#sai:False#kernel_s:5.html
serialized_explanation/KNN#heart#instance:143#num_features:13#feat_selec:lasso_path#discre:decile#sai:True#kernel_s:5.html
serialized_explanation/KNN#heart#instance:143#num_features:13#feat_selec:lasso_path#discre:decile#sai:False#kernel_s:5.html
serialized_explanation/KNN#heart#instance:143#num_features:13#feat_selec:none#discre:quartile#sai:True#kernel_s:5.html
serialized_explanation/KNN#heart#instance:143#num_features:13#feat_selec:none#discre:quartile#sai:False#kernel_s:5.html
serialized_explanation/KNN#heart#instance:143#num_features:13#feat_selec:none#discre:decile#sai:True#kernel_s:5.html
serialized_expl

serialized_explanation/KNN#heart#instance:143#num_features:13#feat_selec:lasso_path#discre:decile#sai:True#kernel_s:7.html
serialized_explanation/KNN#heart#instance:143#num_features:13#feat_selec:lasso_path#discre:decile#sai:False#kernel_s:7.html
serialized_explanation/KNN#heart#instance:143#num_features:13#feat_selec:none#discre:quartile#sai:True#kernel_s:7.html
serialized_explanation/KNN#heart#instance:143#num_features:13#feat_selec:none#discre:quartile#sai:False#kernel_s:7.html
serialized_explanation/KNN#heart#instance:143#num_features:13#feat_selec:none#discre:decile#sai:True#kernel_s:7.html
serialized_explanation/KNN#heart#instance:143#num_features:13#feat_selec:none#discre:decile#sai:False#kernel_s:7.html
serialized_explanation/KNN#heart#instance:143#num_features:13#feat_selec:auto#discre:quartile#sai:True#kernel_s:7.html
serialized_explanation/KNN#heart#instance:143#num_features:13#feat_selec:auto#discre:quartile#sai:False#kernel_s:7.html
serialized_explanation/KNN#heart#instanc

serialized_explanation/KNN#heart#instance:143#num_features:13#feat_selec:none#discre:quartile#sai:False#kernel_s:10.html
serialized_explanation/KNN#heart#instance:143#num_features:13#feat_selec:none#discre:decile#sai:True#kernel_s:10.html
serialized_explanation/KNN#heart#instance:143#num_features:13#feat_selec:none#discre:decile#sai:False#kernel_s:10.html
serialized_explanation/KNN#heart#instance:143#num_features:13#feat_selec:auto#discre:quartile#sai:True#kernel_s:10.html
serialized_explanation/KNN#heart#instance:143#num_features:13#feat_selec:auto#discre:quartile#sai:False#kernel_s:10.html
serialized_explanation/KNN#heart#instance:143#num_features:13#feat_selec:auto#discre:decile#sai:True#kernel_s:10.html
serialized_explanation/KNN#heart#instance:143#num_features:13#feat_selec:auto#discre:decile#sai:False#kernel_s:10.html
serialized_explanation/KNN#heart#instance:203#num_features:13#feat_selec:forward_selection#discre:quartile#sai:True#kernel_s:10.html
serialized_explanation/KNN#hear

### Wine dataset

In [20]:
# import numpy as np
import lime
import lime.lime_tabular
import sys
sys.path.append('./ml_helper')
import ml_helper_train_model as model_train
import ml_helper_serialize_model as serializer
sys.path.append('./explanation_helper')
import expla_helper as explanation
from sklearn.model_selection import train_test_split

wine = pd.read_csv('data/wine_limpo.csv', sep = ',')
# wine.drop(['Unnamed: 0'], axis = 1, inplace = True)
wine = explanation.to_float(wine)

model_file_name = 'serialized_model/KNN_wine2.pkl'
model = serializer.load_model(model_file_name)

dataset = wine.copy(deep = True)
target_names = np.array(['ruim', 'bom'])

folder = 'serialized_explanation/'
model_dataset = model_file_name.replace('serialized_model/','').replace('.pkl', '').replace('_', '#')

kernel_width = [math.ceil(dataset.shape[1]*0.1),
                math.ceil(dataset.shape[1]*0.3),
                math.ceil(dataset.shape[1]*0.5),
                math.ceil(dataset.shape[1]*0.7)]

for size in kernel_width:
    for sample in wine_sample:
        for f_selec in feature_selection:
            for disc in discretizer:
                for sai in sample_around_instance:
                    instance_index = sample
                    num_features = dataset.columns.size - 1 #removing the target

                    path = """{0}{1}#instance:{2}#""".format(folder, model_dataset, instance_index)
                    params_01 = """num_features:{0}#feat_selec:{1}#discre:{2}#""".format(num_features, f_selec, disc)
                    params_02 = """sai:{0}#kernel_s:{1}.html""".format(sai, size)
                    
                    final_file = """{0}{1}{2}""".format(path, params_01, params_02)
                    print(final_file)

                    explanation.explain_it_param(model,
                                               dataset,
                                               target_names,
                                               num_features,
                                               instance_index,
                                               final_file,
                                               #object level
                                               size,
                                               f_selec,     
                                               discretize_continuous,
                                               disc,
                                               sai)


serialized_explanation/KNN#wine2#instance:86#num_features:11#feat_selec:forward_selection#discre:quartile#sai:True#kernel_s:2.html
serialized_explanation/KNN#wine2#instance:86#num_features:11#feat_selec:forward_selection#discre:quartile#sai:False#kernel_s:2.html
serialized_explanation/KNN#wine2#instance:86#num_features:11#feat_selec:forward_selection#discre:decile#sai:True#kernel_s:2.html
serialized_explanation/KNN#wine2#instance:86#num_features:11#feat_selec:forward_selection#discre:decile#sai:False#kernel_s:2.html
serialized_explanation/KNN#wine2#instance:86#num_features:11#feat_selec:lasso_path#discre:quartile#sai:True#kernel_s:2.html
serialized_explanation/KNN#wine2#instance:86#num_features:11#feat_selec:lasso_path#discre:quartile#sai:False#kernel_s:2.html
serialized_explanation/KNN#wine2#instance:86#num_features:11#feat_selec:lasso_path#discre:decile#sai:True#kernel_s:2.html
serialized_explanation/KNN#wine2#instance:86#num_features:11#feat_selec:lasso_path#discre:decile#sai:False#

serialized_explanation/KNN#wine2#instance:86#num_features:11#feat_selec:forward_selection#discre:decile#sai:False#kernel_s:4.html
serialized_explanation/KNN#wine2#instance:86#num_features:11#feat_selec:lasso_path#discre:quartile#sai:True#kernel_s:4.html
serialized_explanation/KNN#wine2#instance:86#num_features:11#feat_selec:lasso_path#discre:quartile#sai:False#kernel_s:4.html
serialized_explanation/KNN#wine2#instance:86#num_features:11#feat_selec:lasso_path#discre:decile#sai:True#kernel_s:4.html
serialized_explanation/KNN#wine2#instance:86#num_features:11#feat_selec:lasso_path#discre:decile#sai:False#kernel_s:4.html
serialized_explanation/KNN#wine2#instance:86#num_features:11#feat_selec:none#discre:quartile#sai:True#kernel_s:4.html
serialized_explanation/KNN#wine2#instance:86#num_features:11#feat_selec:none#discre:quartile#sai:False#kernel_s:4.html
serialized_explanation/KNN#wine2#instance:86#num_features:11#feat_selec:none#discre:decile#sai:True#kernel_s:4.html
serialized_explanation/

serialized_explanation/KNN#wine2#instance:86#num_features:11#feat_selec:lasso_path#discre:decile#sai:True#kernel_s:6.html
serialized_explanation/KNN#wine2#instance:86#num_features:11#feat_selec:lasso_path#discre:decile#sai:False#kernel_s:6.html
serialized_explanation/KNN#wine2#instance:86#num_features:11#feat_selec:none#discre:quartile#sai:True#kernel_s:6.html
serialized_explanation/KNN#wine2#instance:86#num_features:11#feat_selec:none#discre:quartile#sai:False#kernel_s:6.html
serialized_explanation/KNN#wine2#instance:86#num_features:11#feat_selec:none#discre:decile#sai:True#kernel_s:6.html
serialized_explanation/KNN#wine2#instance:86#num_features:11#feat_selec:none#discre:decile#sai:False#kernel_s:6.html
serialized_explanation/KNN#wine2#instance:86#num_features:11#feat_selec:auto#discre:quartile#sai:True#kernel_s:6.html
serialized_explanation/KNN#wine2#instance:86#num_features:11#feat_selec:auto#discre:quartile#sai:False#kernel_s:6.html
serialized_explanation/KNN#wine2#instance:86#num

serialized_explanation/KNN#wine2#instance:86#num_features:11#feat_selec:none#discre:quartile#sai:False#kernel_s:9.html
serialized_explanation/KNN#wine2#instance:86#num_features:11#feat_selec:none#discre:decile#sai:True#kernel_s:9.html
serialized_explanation/KNN#wine2#instance:86#num_features:11#feat_selec:none#discre:decile#sai:False#kernel_s:9.html
serialized_explanation/KNN#wine2#instance:86#num_features:11#feat_selec:auto#discre:quartile#sai:True#kernel_s:9.html
serialized_explanation/KNN#wine2#instance:86#num_features:11#feat_selec:auto#discre:quartile#sai:False#kernel_s:9.html
serialized_explanation/KNN#wine2#instance:86#num_features:11#feat_selec:auto#discre:decile#sai:True#kernel_s:9.html
serialized_explanation/KNN#wine2#instance:86#num_features:11#feat_selec:auto#discre:decile#sai:False#kernel_s:9.html
serialized_explanation/KNN#wine2#instance:1458#num_features:11#feat_selec:forward_selection#discre:quartile#sai:True#kernel_s:9.html
serialized_explanation/KNN#wine2#instance:145

### Diabetes dataset

In [21]:
# import numpy as np
import lime
import lime.lime_tabular
import sys
sys.path.append('./ml_helper')
import ml_helper_train_model as model_train
import ml_helper_serialize_model as serializer
sys.path.append('./explanation_helper')
import expla_helper as explanation
from sklearn.model_selection import train_test_split

diabetes = pd.read_csv('data/diabetes_limpo.csv', sep = ',')
# diabetes.drop(['Unnamed: 0'], axis = 1, inplace = True)
diabetes = explanation.to_float(diabetes)

model_file_name = 'serialized_model/KNN_diabetes2.pkl'
model = serializer.load_model(model_file_name)

dataset = diabetes.copy(deep = True)
target_names = np.array(['saudável', 'doente'])

folder = 'serialized_explanation/'
model_dataset = model_file_name.replace('serialized_model/','').replace('.pkl', '').replace('_', '#')


kernel_width = [math.ceil(dataset.shape[1]*0.1),
                math.ceil(dataset.shape[1]*0.3),
                math.ceil(dataset.shape[1]*0.5),
                math.ceil(dataset.shape[1]*0.7)]

for size in kernel_width:
    for sample in diabetes_sample:
        for f_selec in feature_selection:
            for disc in discretizer:
                for sai in sample_around_instance:
                    instance_index = sample
                    num_features = dataset.columns.size - 1 #removing the target

                    path = """{0}{1}#instance:{2}#""".format(folder, model_dataset, instance_index)
                    params_01 = """num_features:{0}#feat_selec:{1}#discre:{2}#""".format(num_features, f_selec, disc)
                    params_02 = """sai:{0}#kernel_s:{1}.html""".format(sai, size)
                    
                    final_file = """{0}{1}{2}""".format(path, params_01, params_02)
                    print(final_file)

                    explanation.explain_it_param(model,
                                               dataset,
                                               target_names,
                                               num_features,
                                               instance_index,
                                               final_file,
                                               #object level
                                               size,
                                               f_selec,     
                                               discretize_continuous,
                                               disc,
                                               sai)


serialized_explanation/KNN#diabetes2#instance:291#num_features:8#feat_selec:forward_selection#discre:quartile#sai:True#kernel_s:1.html
serialized_explanation/KNN#diabetes2#instance:291#num_features:8#feat_selec:forward_selection#discre:quartile#sai:False#kernel_s:1.html
serialized_explanation/KNN#diabetes2#instance:291#num_features:8#feat_selec:forward_selection#discre:decile#sai:True#kernel_s:1.html
serialized_explanation/KNN#diabetes2#instance:291#num_features:8#feat_selec:forward_selection#discre:decile#sai:False#kernel_s:1.html
serialized_explanation/KNN#diabetes2#instance:291#num_features:8#feat_selec:lasso_path#discre:quartile#sai:True#kernel_s:1.html
serialized_explanation/KNN#diabetes2#instance:291#num_features:8#feat_selec:lasso_path#discre:quartile#sai:False#kernel_s:1.html
serialized_explanation/KNN#diabetes2#instance:291#num_features:8#feat_selec:lasso_path#discre:decile#sai:True#kernel_s:1.html
serialized_explanation/KNN#diabetes2#instance:291#num_features:8#feat_selec:las

serialized_explanation/KNN#diabetes2#instance:291#num_features:8#feat_selec:forward_selection#discre:quartile#sai:False#kernel_s:3.html
serialized_explanation/KNN#diabetes2#instance:291#num_features:8#feat_selec:forward_selection#discre:decile#sai:True#kernel_s:3.html
serialized_explanation/KNN#diabetes2#instance:291#num_features:8#feat_selec:forward_selection#discre:decile#sai:False#kernel_s:3.html
serialized_explanation/KNN#diabetes2#instance:291#num_features:8#feat_selec:lasso_path#discre:quartile#sai:True#kernel_s:3.html
serialized_explanation/KNN#diabetes2#instance:291#num_features:8#feat_selec:lasso_path#discre:quartile#sai:False#kernel_s:3.html
serialized_explanation/KNN#diabetes2#instance:291#num_features:8#feat_selec:lasso_path#discre:decile#sai:True#kernel_s:3.html
serialized_explanation/KNN#diabetes2#instance:291#num_features:8#feat_selec:lasso_path#discre:decile#sai:False#kernel_s:3.html
serialized_explanation/KNN#diabetes2#instance:291#num_features:8#feat_selec:none#discre

serialized_explanation/KNN#diabetes2#instance:291#num_features:8#feat_selec:forward_selection#discre:decile#sai:True#kernel_s:5.html
serialized_explanation/KNN#diabetes2#instance:291#num_features:8#feat_selec:forward_selection#discre:decile#sai:False#kernel_s:5.html
serialized_explanation/KNN#diabetes2#instance:291#num_features:8#feat_selec:lasso_path#discre:quartile#sai:True#kernel_s:5.html
serialized_explanation/KNN#diabetes2#instance:291#num_features:8#feat_selec:lasso_path#discre:quartile#sai:False#kernel_s:5.html
serialized_explanation/KNN#diabetes2#instance:291#num_features:8#feat_selec:lasso_path#discre:decile#sai:True#kernel_s:5.html
serialized_explanation/KNN#diabetes2#instance:291#num_features:8#feat_selec:lasso_path#discre:decile#sai:False#kernel_s:5.html
serialized_explanation/KNN#diabetes2#instance:291#num_features:8#feat_selec:none#discre:quartile#sai:True#kernel_s:5.html
serialized_explanation/KNN#diabetes2#instance:291#num_features:8#feat_selec:none#discre:quartile#sai:

serialized_explanation/KNN#diabetes2#instance:291#num_features:8#feat_selec:forward_selection#discre:decile#sai:False#kernel_s:7.html
serialized_explanation/KNN#diabetes2#instance:291#num_features:8#feat_selec:lasso_path#discre:quartile#sai:True#kernel_s:7.html
serialized_explanation/KNN#diabetes2#instance:291#num_features:8#feat_selec:lasso_path#discre:quartile#sai:False#kernel_s:7.html
serialized_explanation/KNN#diabetes2#instance:291#num_features:8#feat_selec:lasso_path#discre:decile#sai:True#kernel_s:7.html
serialized_explanation/KNN#diabetes2#instance:291#num_features:8#feat_selec:lasso_path#discre:decile#sai:False#kernel_s:7.html
serialized_explanation/KNN#diabetes2#instance:291#num_features:8#feat_selec:none#discre:quartile#sai:True#kernel_s:7.html
serialized_explanation/KNN#diabetes2#instance:291#num_features:8#feat_selec:none#discre:quartile#sai:False#kernel_s:7.html
serialized_explanation/KNN#diabetes2#instance:291#num_features:8#feat_selec:none#discre:decile#sai:True#kernel_

## Gaussian Process

### Heart dataset

In [22]:
# import numpy as np
import lime
import lime.lime_tabular
import sys
sys.path.append('./ml_helper')
import ml_helper_train_model as model_train
import ml_helper_serialize_model as serializer
sys.path.append('./explanation_helper')
import expla_helper as explanation
from sklearn.model_selection import train_test_split

heart = pd.read_csv('data/datasets_33180_43520_heart.csv', sep = ',')
heart = explanation.to_float(heart)

model_file_name = 'serialized_model/GaussianProcessClassifier_heart.pkl'
model = serializer.load_model(model_file_name)

dataset = heart.copy(deep = True)
target_names = np.array(['saudável', 'doente'])

folder = 'serialized_explanation/'
model_dataset = model_file_name.replace('serialized_model/','').replace('.pkl', '').replace('_', '#')

kernel_width = [math.ceil(dataset.shape[1]*0.1),
                math.ceil(dataset.shape[1]*0.3),
                math.ceil(dataset.shape[1]*0.5),
                math.ceil(dataset.shape[1]*0.7)]

for size in kernel_width:
    for sample in heart_sample:
        for f_selec in feature_selection:
            for disc in discretizer:
                for sai in sample_around_instance:
                    instance_index = sample
                    num_features = dataset.columns.size - 1 #removing the target

                    path = """{0}{1}#instance:{2}#""".format(folder, model_dataset, instance_index)
                    params_01 = """num_features:{0}#feat_selec:{1}#discre:{2}#""".format(num_features, f_selec, disc)
                    params_02 = """sai:{0}#kernel_s:{1}.html""".format(sai, size)
                    
                    final_file = """{0}{1}{2}""".format(path, params_01, params_02)
                    print(final_file)

                    explanation.explain_it_param(model,
                                               dataset,
                                               target_names,
                                               num_features,
                                               instance_index,
                                               final_file,
                                               #object level
                                               size,
                                               f_selec,     
                                               discretize_continuous,
                                               disc,
                                               sai)


serialized_explanation/GaussianProcessClassifier#heart#instance:143#num_features:13#feat_selec:forward_selection#discre:quartile#sai:True#kernel_s:2.html
serialized_explanation/GaussianProcessClassifier#heart#instance:143#num_features:13#feat_selec:forward_selection#discre:quartile#sai:False#kernel_s:2.html
serialized_explanation/GaussianProcessClassifier#heart#instance:143#num_features:13#feat_selec:forward_selection#discre:decile#sai:True#kernel_s:2.html
serialized_explanation/GaussianProcessClassifier#heart#instance:143#num_features:13#feat_selec:forward_selection#discre:decile#sai:False#kernel_s:2.html
serialized_explanation/GaussianProcessClassifier#heart#instance:143#num_features:13#feat_selec:lasso_path#discre:quartile#sai:True#kernel_s:2.html
serialized_explanation/GaussianProcessClassifier#heart#instance:143#num_features:13#feat_selec:lasso_path#discre:quartile#sai:False#kernel_s:2.html
serialized_explanation/GaussianProcessClassifier#heart#instance:143#num_features:13#feat_se

serialized_explanation/GaussianProcessClassifier#heart#instance:96#num_features:13#feat_selec:none#discre:quartile#sai:False#kernel_s:2.html
serialized_explanation/GaussianProcessClassifier#heart#instance:96#num_features:13#feat_selec:none#discre:decile#sai:True#kernel_s:2.html
serialized_explanation/GaussianProcessClassifier#heart#instance:96#num_features:13#feat_selec:none#discre:decile#sai:False#kernel_s:2.html
serialized_explanation/GaussianProcessClassifier#heart#instance:96#num_features:13#feat_selec:auto#discre:quartile#sai:True#kernel_s:2.html
serialized_explanation/GaussianProcessClassifier#heart#instance:96#num_features:13#feat_selec:auto#discre:quartile#sai:False#kernel_s:2.html
serialized_explanation/GaussianProcessClassifier#heart#instance:96#num_features:13#feat_selec:auto#discre:decile#sai:True#kernel_s:2.html
serialized_explanation/GaussianProcessClassifier#heart#instance:96#num_features:13#feat_selec:auto#discre:decile#sai:False#kernel_s:2.html
serialized_explanation/G

serialized_explanation/GaussianProcessClassifier#heart#instance:96#num_features:13#feat_selec:forward_selection#discre:decile#sai:True#kernel_s:5.html
serialized_explanation/GaussianProcessClassifier#heart#instance:96#num_features:13#feat_selec:forward_selection#discre:decile#sai:False#kernel_s:5.html
serialized_explanation/GaussianProcessClassifier#heart#instance:96#num_features:13#feat_selec:lasso_path#discre:quartile#sai:True#kernel_s:5.html
serialized_explanation/GaussianProcessClassifier#heart#instance:96#num_features:13#feat_selec:lasso_path#discre:quartile#sai:False#kernel_s:5.html
serialized_explanation/GaussianProcessClassifier#heart#instance:96#num_features:13#feat_selec:lasso_path#discre:decile#sai:True#kernel_s:5.html
serialized_explanation/GaussianProcessClassifier#heart#instance:96#num_features:13#feat_selec:lasso_path#discre:decile#sai:False#kernel_s:5.html
serialized_explanation/GaussianProcessClassifier#heart#instance:96#num_features:13#feat_selec:none#discre:quartile#

serialized_explanation/GaussianProcessClassifier#heart#instance:242#num_features:13#feat_selec:none#discre:decile#sai:False#kernel_s:7.html
serialized_explanation/GaussianProcessClassifier#heart#instance:242#num_features:13#feat_selec:auto#discre:quartile#sai:True#kernel_s:7.html
serialized_explanation/GaussianProcessClassifier#heart#instance:242#num_features:13#feat_selec:auto#discre:quartile#sai:False#kernel_s:7.html
serialized_explanation/GaussianProcessClassifier#heart#instance:242#num_features:13#feat_selec:auto#discre:decile#sai:True#kernel_s:7.html
serialized_explanation/GaussianProcessClassifier#heart#instance:242#num_features:13#feat_selec:auto#discre:decile#sai:False#kernel_s:7.html
serialized_explanation/GaussianProcessClassifier#heart#instance:96#num_features:13#feat_selec:forward_selection#discre:quartile#sai:True#kernel_s:7.html
serialized_explanation/GaussianProcessClassifier#heart#instance:96#num_features:13#feat_selec:forward_selection#discre:quartile#sai:False#kernel_

serialized_explanation/GaussianProcessClassifier#heart#instance:242#num_features:13#feat_selec:lasso_path#discre:quartile#sai:True#kernel_s:10.html
serialized_explanation/GaussianProcessClassifier#heart#instance:242#num_features:13#feat_selec:lasso_path#discre:quartile#sai:False#kernel_s:10.html
serialized_explanation/GaussianProcessClassifier#heart#instance:242#num_features:13#feat_selec:lasso_path#discre:decile#sai:True#kernel_s:10.html
serialized_explanation/GaussianProcessClassifier#heart#instance:242#num_features:13#feat_selec:lasso_path#discre:decile#sai:False#kernel_s:10.html
serialized_explanation/GaussianProcessClassifier#heart#instance:242#num_features:13#feat_selec:none#discre:quartile#sai:True#kernel_s:10.html
serialized_explanation/GaussianProcessClassifier#heart#instance:242#num_features:13#feat_selec:none#discre:quartile#sai:False#kernel_s:10.html
serialized_explanation/GaussianProcessClassifier#heart#instance:242#num_features:13#feat_selec:none#discre:decile#sai:True#ke

### Wine dataset

In [23]:
# import numpy as np
import lime
import lime.lime_tabular
import sys
sys.path.append('./ml_helper')
import ml_helper_train_model as model_train
import ml_helper_serialize_model as serializer
sys.path.append('./explanation_helper')
import expla_helper as explanation
from sklearn.model_selection import train_test_split

wine = pd.read_csv('data/wine_limpo.csv', sep = ',')
# wine.drop(['Unnamed: 0'], axis = 1, inplace = True)
wine = explanation.to_float(wine)

model_file_name = 'serialized_model/GaussianProcessClassifier_wine2.pkl'
model = serializer.load_model(model_file_name)

dataset = wine.copy(deep = True)
target_names = np.array(['ruim', 'bom'])

folder = 'serialized_explanation/'
model_dataset = model_file_name.replace('serialized_model/','').replace('.pkl', '').replace('_', '#')

kernel_width = [math.ceil(dataset.shape[1]*0.1),
                math.ceil(dataset.shape[1]*0.3),
                math.ceil(dataset.shape[1]*0.5),
                math.ceil(dataset.shape[1]*0.7)]

for size in kernel_width:
    for sample in wine_sample:
        for f_selec in feature_selection:
            for disc in discretizer:
                for sai in sample_around_instance:
                    instance_index = sample
                    num_features = dataset.columns.size - 1 #removing the target

                    path = """{0}{1}#instance:{2}#""".format(folder, model_dataset, instance_index)
                    params_01 = """num_features:{0}#feat_selec:{1}#discre:{2}#""".format(num_features, f_selec, disc)
                    params_02 = """sai:{0}#kernel_s:{1}.html""".format(sai, size)
                    
                    final_file = """{0}{1}{2}""".format(path, params_01, params_02)
                    print(final_file)

                    explanation.explain_it_param(model,
                                               dataset,
                                               target_names,
                                               num_features,
                                               instance_index,
                                               final_file,
                                               #object level
                                               size,
                                               f_selec,     
                                               discretize_continuous,
                                               disc,
                                               sai)


serialized_explanation/GaussianProcessClassifier#wine2#instance:86#num_features:11#feat_selec:forward_selection#discre:quartile#sai:True#kernel_s:2.html
serialized_explanation/GaussianProcessClassifier#wine2#instance:86#num_features:11#feat_selec:forward_selection#discre:quartile#sai:False#kernel_s:2.html
serialized_explanation/GaussianProcessClassifier#wine2#instance:86#num_features:11#feat_selec:forward_selection#discre:decile#sai:True#kernel_s:2.html
serialized_explanation/GaussianProcessClassifier#wine2#instance:86#num_features:11#feat_selec:forward_selection#discre:decile#sai:False#kernel_s:2.html
serialized_explanation/GaussianProcessClassifier#wine2#instance:86#num_features:11#feat_selec:lasso_path#discre:quartile#sai:True#kernel_s:2.html
serialized_explanation/GaussianProcessClassifier#wine2#instance:86#num_features:11#feat_selec:lasso_path#discre:quartile#sai:False#kernel_s:2.html
serialized_explanation/GaussianProcessClassifier#wine2#instance:86#num_features:11#feat_selec:las

serialized_explanation/GaussianProcessClassifier#wine2#instance:1003#num_features:11#feat_selec:none#discre:quartile#sai:False#kernel_s:2.html
serialized_explanation/GaussianProcessClassifier#wine2#instance:1003#num_features:11#feat_selec:none#discre:decile#sai:True#kernel_s:2.html
serialized_explanation/GaussianProcessClassifier#wine2#instance:1003#num_features:11#feat_selec:none#discre:decile#sai:False#kernel_s:2.html
serialized_explanation/GaussianProcessClassifier#wine2#instance:1003#num_features:11#feat_selec:auto#discre:quartile#sai:True#kernel_s:2.html
serialized_explanation/GaussianProcessClassifier#wine2#instance:1003#num_features:11#feat_selec:auto#discre:quartile#sai:False#kernel_s:2.html
serialized_explanation/GaussianProcessClassifier#wine2#instance:1003#num_features:11#feat_selec:auto#discre:decile#sai:True#kernel_s:2.html
serialized_explanation/GaussianProcessClassifier#wine2#instance:1003#num_features:11#feat_selec:auto#discre:decile#sai:False#kernel_s:2.html
serialized

serialized_explanation/GaussianProcessClassifier#wine2#instance:1003#num_features:11#feat_selec:forward_selection#discre:decile#sai:True#kernel_s:4.html
serialized_explanation/GaussianProcessClassifier#wine2#instance:1003#num_features:11#feat_selec:forward_selection#discre:decile#sai:False#kernel_s:4.html
serialized_explanation/GaussianProcessClassifier#wine2#instance:1003#num_features:11#feat_selec:lasso_path#discre:quartile#sai:True#kernel_s:4.html
serialized_explanation/GaussianProcessClassifier#wine2#instance:1003#num_features:11#feat_selec:lasso_path#discre:quartile#sai:False#kernel_s:4.html
serialized_explanation/GaussianProcessClassifier#wine2#instance:1003#num_features:11#feat_selec:lasso_path#discre:decile#sai:True#kernel_s:4.html
serialized_explanation/GaussianProcessClassifier#wine2#instance:1003#num_features:11#feat_selec:lasso_path#discre:decile#sai:False#kernel_s:4.html
serialized_explanation/GaussianProcessClassifier#wine2#instance:1003#num_features:11#feat_selec:none#di

serialized_explanation/GaussianProcessClassifier#wine2#instance:791#num_features:11#feat_selec:none#discre:decile#sai:False#kernel_s:6.html
serialized_explanation/GaussianProcessClassifier#wine2#instance:791#num_features:11#feat_selec:auto#discre:quartile#sai:True#kernel_s:6.html
serialized_explanation/GaussianProcessClassifier#wine2#instance:791#num_features:11#feat_selec:auto#discre:quartile#sai:False#kernel_s:6.html
serialized_explanation/GaussianProcessClassifier#wine2#instance:791#num_features:11#feat_selec:auto#discre:decile#sai:True#kernel_s:6.html
serialized_explanation/GaussianProcessClassifier#wine2#instance:791#num_features:11#feat_selec:auto#discre:decile#sai:False#kernel_s:6.html
serialized_explanation/GaussianProcessClassifier#wine2#instance:1003#num_features:11#feat_selec:forward_selection#discre:quartile#sai:True#kernel_s:6.html
serialized_explanation/GaussianProcessClassifier#wine2#instance:1003#num_features:11#feat_selec:forward_selection#discre:quartile#sai:False#ker

serialized_explanation/GaussianProcessClassifier#wine2#instance:791#num_features:11#feat_selec:lasso_path#discre:quartile#sai:True#kernel_s:9.html
serialized_explanation/GaussianProcessClassifier#wine2#instance:791#num_features:11#feat_selec:lasso_path#discre:quartile#sai:False#kernel_s:9.html
serialized_explanation/GaussianProcessClassifier#wine2#instance:791#num_features:11#feat_selec:lasso_path#discre:decile#sai:True#kernel_s:9.html
serialized_explanation/GaussianProcessClassifier#wine2#instance:791#num_features:11#feat_selec:lasso_path#discre:decile#sai:False#kernel_s:9.html
serialized_explanation/GaussianProcessClassifier#wine2#instance:791#num_features:11#feat_selec:none#discre:quartile#sai:True#kernel_s:9.html
serialized_explanation/GaussianProcessClassifier#wine2#instance:791#num_features:11#feat_selec:none#discre:quartile#sai:False#kernel_s:9.html
serialized_explanation/GaussianProcessClassifier#wine2#instance:791#num_features:11#feat_selec:none#discre:decile#sai:True#kernel_s

### Diabetes dataset

In [24]:
# import numpy as np
import lime
import lime.lime_tabular
import sys
sys.path.append('./ml_helper')
import ml_helper_train_model as model_train
import ml_helper_serialize_model as serializer
sys.path.append('./explanation_helper')
import expla_helper as explanation
from sklearn.model_selection import train_test_split

diabetes = pd.read_csv('data/diabetes_limpo.csv', sep = ',')
# diabetes.drop(['Unnamed: 0'], axis = 1, inplace = True)
diabetes = explanation.to_float(diabetes)

model_file_name = 'serialized_model/GaussianProcessClassifier_diabetes2.pkl'
model = serializer.load_model(model_file_name)

dataset = diabetes.copy(deep = True)
target_names = np.array(['saudável', 'doente'])

folder = 'serialized_explanation/'
model_dataset = model_file_name.replace('serialized_model/','').replace('.pkl', '').replace('_', '#')


kernel_width = [math.ceil(dataset.shape[1]*0.1),
                math.ceil(dataset.shape[1]*0.3),
                math.ceil(dataset.shape[1]*0.5),
                math.ceil(dataset.shape[1]*0.7)]

for size in kernel_width:
    for sample in diabetes_sample:
        for f_selec in feature_selection:
            for disc in discretizer:
                for sai in sample_around_instance:
                    instance_index = sample
                    num_features = dataset.columns.size - 1 #removing the target

                    path = """{0}{1}#instance:{2}#""".format(folder, model_dataset, instance_index)
                    params_01 = """num_features:{0}#feat_selec:{1}#discre:{2}#""".format(num_features, f_selec, disc)
                    params_02 = """sai:{0}#kernel_s:{1}.html""".format(sai, size)
                    
                    final_file = """{0}{1}{2}""".format(path, params_01, params_02)
                    print(final_file)

                    explanation.explain_it_param(model,
                                               dataset,
                                               target_names,
                                               num_features,
                                               instance_index,
                                               final_file,
                                               #object level
                                               size,
                                               f_selec,     
                                               discretize_continuous,
                                               disc,
                                               sai)


serialized_explanation/GaussianProcessClassifier#diabetes2#instance:291#num_features:8#feat_selec:forward_selection#discre:quartile#sai:True#kernel_s:1.html
serialized_explanation/GaussianProcessClassifier#diabetes2#instance:291#num_features:8#feat_selec:forward_selection#discre:quartile#sai:False#kernel_s:1.html
serialized_explanation/GaussianProcessClassifier#diabetes2#instance:291#num_features:8#feat_selec:forward_selection#discre:decile#sai:True#kernel_s:1.html
serialized_explanation/GaussianProcessClassifier#diabetes2#instance:291#num_features:8#feat_selec:forward_selection#discre:decile#sai:False#kernel_s:1.html
serialized_explanation/GaussianProcessClassifier#diabetes2#instance:291#num_features:8#feat_selec:lasso_path#discre:quartile#sai:True#kernel_s:1.html
serialized_explanation/GaussianProcessClassifier#diabetes2#instance:291#num_features:8#feat_selec:lasso_path#discre:quartile#sai:False#kernel_s:1.html
serialized_explanation/GaussianProcessClassifier#diabetes2#instance:291#n

serialized_explanation/GaussianProcessClassifier#diabetes2#instance:755#num_features:8#feat_selec:none#discre:quartile#sai:True#kernel_s:1.html
serialized_explanation/GaussianProcessClassifier#diabetes2#instance:755#num_features:8#feat_selec:none#discre:quartile#sai:False#kernel_s:1.html
serialized_explanation/GaussianProcessClassifier#diabetes2#instance:755#num_features:8#feat_selec:none#discre:decile#sai:True#kernel_s:1.html
serialized_explanation/GaussianProcessClassifier#diabetes2#instance:755#num_features:8#feat_selec:none#discre:decile#sai:False#kernel_s:1.html
serialized_explanation/GaussianProcessClassifier#diabetes2#instance:755#num_features:8#feat_selec:auto#discre:quartile#sai:True#kernel_s:1.html
serialized_explanation/GaussianProcessClassifier#diabetes2#instance:755#num_features:8#feat_selec:auto#discre:quartile#sai:False#kernel_s:1.html
serialized_explanation/GaussianProcessClassifier#diabetes2#instance:755#num_features:8#feat_selec:auto#discre:decile#sai:True#kernel_s:1.

serialized_explanation/GaussianProcessClassifier#diabetes2#instance:755#num_features:8#feat_selec:forward_selection#discre:quartile#sai:True#kernel_s:3.html
serialized_explanation/GaussianProcessClassifier#diabetes2#instance:755#num_features:8#feat_selec:forward_selection#discre:quartile#sai:False#kernel_s:3.html
serialized_explanation/GaussianProcessClassifier#diabetes2#instance:755#num_features:8#feat_selec:forward_selection#discre:decile#sai:True#kernel_s:3.html
serialized_explanation/GaussianProcessClassifier#diabetes2#instance:755#num_features:8#feat_selec:forward_selection#discre:decile#sai:False#kernel_s:3.html
serialized_explanation/GaussianProcessClassifier#diabetes2#instance:755#num_features:8#feat_selec:lasso_path#discre:quartile#sai:True#kernel_s:3.html
serialized_explanation/GaussianProcessClassifier#diabetes2#instance:755#num_features:8#feat_selec:lasso_path#discre:quartile#sai:False#kernel_s:3.html
serialized_explanation/GaussianProcessClassifier#diabetes2#instance:755#n

serialized_explanation/GaussianProcessClassifier#diabetes2#instance:60#num_features:8#feat_selec:none#discre:quartile#sai:True#kernel_s:5.html
serialized_explanation/GaussianProcessClassifier#diabetes2#instance:60#num_features:8#feat_selec:none#discre:quartile#sai:False#kernel_s:5.html
serialized_explanation/GaussianProcessClassifier#diabetes2#instance:60#num_features:8#feat_selec:none#discre:decile#sai:True#kernel_s:5.html
serialized_explanation/GaussianProcessClassifier#diabetes2#instance:60#num_features:8#feat_selec:none#discre:decile#sai:False#kernel_s:5.html
serialized_explanation/GaussianProcessClassifier#diabetes2#instance:60#num_features:8#feat_selec:auto#discre:quartile#sai:True#kernel_s:5.html
serialized_explanation/GaussianProcessClassifier#diabetes2#instance:60#num_features:8#feat_selec:auto#discre:quartile#sai:False#kernel_s:5.html
serialized_explanation/GaussianProcessClassifier#diabetes2#instance:60#num_features:8#feat_selec:auto#discre:decile#sai:True#kernel_s:5.html
se

serialized_explanation/GaussianProcessClassifier#diabetes2#instance:60#num_features:8#feat_selec:forward_selection#discre:quartile#sai:True#kernel_s:7.html
serialized_explanation/GaussianProcessClassifier#diabetes2#instance:60#num_features:8#feat_selec:forward_selection#discre:quartile#sai:False#kernel_s:7.html
serialized_explanation/GaussianProcessClassifier#diabetes2#instance:60#num_features:8#feat_selec:forward_selection#discre:decile#sai:True#kernel_s:7.html
serialized_explanation/GaussianProcessClassifier#diabetes2#instance:60#num_features:8#feat_selec:forward_selection#discre:decile#sai:False#kernel_s:7.html
serialized_explanation/GaussianProcessClassifier#diabetes2#instance:60#num_features:8#feat_selec:lasso_path#discre:quartile#sai:True#kernel_s:7.html
serialized_explanation/GaussianProcessClassifier#diabetes2#instance:60#num_features:8#feat_selec:lasso_path#discre:quartile#sai:False#kernel_s:7.html
serialized_explanation/GaussianProcessClassifier#diabetes2#instance:60#num_feat

## SVM

### Heart dataset

In [25]:
# import numpy as np
import lime
import lime.lime_tabular
import sys
sys.path.append('./ml_helper')
import ml_helper_train_model as model_train
import ml_helper_serialize_model as serializer
sys.path.append('./explanation_helper')
import expla_helper as explanation
from sklearn.model_selection import train_test_split

heart = pd.read_csv('data/datasets_33180_43520_heart.csv', sep = ',')
heart = explanation.to_float(heart)

model_file_name = 'serialized_model/svm_heart.pkl'
model = serializer.load_model(model_file_name)

dataset = heart.copy(deep = True)
target_names = np.array(['saudável', 'doente'])

folder = 'serialized_explanation/'
model_dataset = model_file_name.replace('serialized_model/','').replace('.pkl', '').replace('_', '#')


kernel_width = [math.ceil(dataset.shape[1]*0.1),
                math.ceil(dataset.shape[1]*0.3),
                math.ceil(dataset.shape[1]*0.5),
                math.ceil(dataset.shape[1]*0.7)]

for size in kernel_width:
    for sample in heart_sample:
        for f_selec in feature_selection:
            for disc in discretizer:
                for sai in sample_around_instance:
                    instance_index = sample
                    num_features = dataset.columns.size - 1 #removing the target

                    path = """{0}{1}#instance:{2}#""".format(folder, model_dataset, instance_index)
                    params_01 = """num_features:{0}#feat_selec:{1}#discre:{2}#""".format(num_features, f_selec, disc)
                    params_02 = """sai:{0}#kernel_s:{1}.html""".format(sai, size)
                    
                    final_file = """{0}{1}{2}""".format(path, params_01, params_02)
                    print(final_file)

                    explanation.explain_it_param(model,
                                               dataset,
                                               target_names,
                                               num_features,
                                               instance_index,
                                               final_file,
                                               #object level
                                               size,
                                               f_selec,     
                                               discretize_continuous,
                                               disc,
                                               sai)


serialized_explanation/svm#heart#instance:143#num_features:13#feat_selec:forward_selection#discre:quartile#sai:True#kernel_s:2.html
serialized_explanation/svm#heart#instance:143#num_features:13#feat_selec:forward_selection#discre:quartile#sai:False#kernel_s:2.html
serialized_explanation/svm#heart#instance:143#num_features:13#feat_selec:forward_selection#discre:decile#sai:True#kernel_s:2.html
serialized_explanation/svm#heart#instance:143#num_features:13#feat_selec:forward_selection#discre:decile#sai:False#kernel_s:2.html
serialized_explanation/svm#heart#instance:143#num_features:13#feat_selec:lasso_path#discre:quartile#sai:True#kernel_s:2.html
serialized_explanation/svm#heart#instance:143#num_features:13#feat_selec:lasso_path#discre:quartile#sai:False#kernel_s:2.html
serialized_explanation/svm#heart#instance:143#num_features:13#feat_selec:lasso_path#discre:decile#sai:True#kernel_s:2.html
serialized_explanation/svm#heart#instance:143#num_features:13#feat_selec:lasso_path#discre:decile#sa

serialized_explanation/svm#heart#instance:143#num_features:13#feat_selec:forward_selection#discre:decile#sai:False#kernel_s:5.html
serialized_explanation/svm#heart#instance:143#num_features:13#feat_selec:lasso_path#discre:quartile#sai:True#kernel_s:5.html
serialized_explanation/svm#heart#instance:143#num_features:13#feat_selec:lasso_path#discre:quartile#sai:False#kernel_s:5.html
serialized_explanation/svm#heart#instance:143#num_features:13#feat_selec:lasso_path#discre:decile#sai:True#kernel_s:5.html
serialized_explanation/svm#heart#instance:143#num_features:13#feat_selec:lasso_path#discre:decile#sai:False#kernel_s:5.html
serialized_explanation/svm#heart#instance:143#num_features:13#feat_selec:none#discre:quartile#sai:True#kernel_s:5.html
serialized_explanation/svm#heart#instance:143#num_features:13#feat_selec:none#discre:quartile#sai:False#kernel_s:5.html
serialized_explanation/svm#heart#instance:143#num_features:13#feat_selec:none#discre:decile#sai:True#kernel_s:5.html
serialized_expl

serialized_explanation/svm#heart#instance:143#num_features:13#feat_selec:lasso_path#discre:decile#sai:True#kernel_s:7.html
serialized_explanation/svm#heart#instance:143#num_features:13#feat_selec:lasso_path#discre:decile#sai:False#kernel_s:7.html
serialized_explanation/svm#heart#instance:143#num_features:13#feat_selec:none#discre:quartile#sai:True#kernel_s:7.html
serialized_explanation/svm#heart#instance:143#num_features:13#feat_selec:none#discre:quartile#sai:False#kernel_s:7.html
serialized_explanation/svm#heart#instance:143#num_features:13#feat_selec:none#discre:decile#sai:True#kernel_s:7.html
serialized_explanation/svm#heart#instance:143#num_features:13#feat_selec:none#discre:decile#sai:False#kernel_s:7.html
serialized_explanation/svm#heart#instance:143#num_features:13#feat_selec:auto#discre:quartile#sai:True#kernel_s:7.html
serialized_explanation/svm#heart#instance:143#num_features:13#feat_selec:auto#discre:quartile#sai:False#kernel_s:7.html
serialized_explanation/svm#heart#instanc

serialized_explanation/svm#heart#instance:143#num_features:13#feat_selec:none#discre:quartile#sai:False#kernel_s:10.html
serialized_explanation/svm#heart#instance:143#num_features:13#feat_selec:none#discre:decile#sai:True#kernel_s:10.html
serialized_explanation/svm#heart#instance:143#num_features:13#feat_selec:none#discre:decile#sai:False#kernel_s:10.html
serialized_explanation/svm#heart#instance:143#num_features:13#feat_selec:auto#discre:quartile#sai:True#kernel_s:10.html
serialized_explanation/svm#heart#instance:143#num_features:13#feat_selec:auto#discre:quartile#sai:False#kernel_s:10.html
serialized_explanation/svm#heart#instance:143#num_features:13#feat_selec:auto#discre:decile#sai:True#kernel_s:10.html
serialized_explanation/svm#heart#instance:143#num_features:13#feat_selec:auto#discre:decile#sai:False#kernel_s:10.html
serialized_explanation/svm#heart#instance:203#num_features:13#feat_selec:forward_selection#discre:quartile#sai:True#kernel_s:10.html
serialized_explanation/svm#hear

### Wine dataset

In [26]:
# import numpy as np
import lime
import lime.lime_tabular
import sys
sys.path.append('./ml_helper')
import ml_helper_train_model as model_train
import ml_helper_serialize_model as serializer
sys.path.append('./explanation_helper')
import expla_helper as explanation
from sklearn.model_selection import train_test_split

wine = pd.read_csv('data/wine_limpo.csv', sep = ',')
# wine.drop(['Unnamed: 0'], axis = 1, inplace = True)
wine = explanation.to_float(wine)

model_file_name = 'serialized_model/svm_wine2.pkl'
model = serializer.load_model(model_file_name)

dataset = wine.copy(deep = True)
target_names = np.array(['ruim', 'bom'])

folder = 'serialized_explanation/'
model_dataset = model_file_name.replace('serialized_model/','').replace('.pkl', '').replace('_', '#')


kernel_width = [math.ceil(dataset.shape[1]*0.1),
                math.ceil(dataset.shape[1]*0.3),
                math.ceil(dataset.shape[1]*0.5),
                math.ceil(dataset.shape[1]*0.7)]

for size in kernel_width:
    for sample in wine_sample:
        for f_selec in feature_selection:
            for disc in discretizer:
                for sai in sample_around_instance:
                    instance_index = sample
                    num_features = dataset.columns.size - 1 #removing the target

                    path = """{0}{1}#instance:{2}#""".format(folder, model_dataset, instance_index)
                    params_01 = """num_features:{0}#feat_selec:{1}#discre:{2}#""".format(num_features, f_selec, disc)
                    params_02 = """sai:{0}#kernel_s:{1}.html""".format(sai, size)
                    
                    final_file = """{0}{1}{2}""".format(path, params_01, params_02)
                    print(final_file)

                    explanation.explain_it_param(model,
                                               dataset,
                                               target_names,
                                               num_features,
                                               instance_index,
                                               final_file,
                                               #object level
                                               size,
                                               f_selec,     
                                               discretize_continuous,
                                               disc,
                                               sai)


serialized_explanation/svm#wine2#instance:86#num_features:11#feat_selec:forward_selection#discre:quartile#sai:True#kernel_s:2.html
serialized_explanation/svm#wine2#instance:86#num_features:11#feat_selec:forward_selection#discre:quartile#sai:False#kernel_s:2.html
serialized_explanation/svm#wine2#instance:86#num_features:11#feat_selec:forward_selection#discre:decile#sai:True#kernel_s:2.html
serialized_explanation/svm#wine2#instance:86#num_features:11#feat_selec:forward_selection#discre:decile#sai:False#kernel_s:2.html
serialized_explanation/svm#wine2#instance:86#num_features:11#feat_selec:lasso_path#discre:quartile#sai:True#kernel_s:2.html
serialized_explanation/svm#wine2#instance:86#num_features:11#feat_selec:lasso_path#discre:quartile#sai:False#kernel_s:2.html
serialized_explanation/svm#wine2#instance:86#num_features:11#feat_selec:lasso_path#discre:decile#sai:True#kernel_s:2.html
serialized_explanation/svm#wine2#instance:86#num_features:11#feat_selec:lasso_path#discre:decile#sai:False#

serialized_explanation/svm#wine2#instance:86#num_features:11#feat_selec:forward_selection#discre:decile#sai:False#kernel_s:4.html
serialized_explanation/svm#wine2#instance:86#num_features:11#feat_selec:lasso_path#discre:quartile#sai:True#kernel_s:4.html
serialized_explanation/svm#wine2#instance:86#num_features:11#feat_selec:lasso_path#discre:quartile#sai:False#kernel_s:4.html
serialized_explanation/svm#wine2#instance:86#num_features:11#feat_selec:lasso_path#discre:decile#sai:True#kernel_s:4.html
serialized_explanation/svm#wine2#instance:86#num_features:11#feat_selec:lasso_path#discre:decile#sai:False#kernel_s:4.html
serialized_explanation/svm#wine2#instance:86#num_features:11#feat_selec:none#discre:quartile#sai:True#kernel_s:4.html
serialized_explanation/svm#wine2#instance:86#num_features:11#feat_selec:none#discre:quartile#sai:False#kernel_s:4.html
serialized_explanation/svm#wine2#instance:86#num_features:11#feat_selec:none#discre:decile#sai:True#kernel_s:4.html
serialized_explanation/

serialized_explanation/svm#wine2#instance:86#num_features:11#feat_selec:lasso_path#discre:decile#sai:True#kernel_s:6.html
serialized_explanation/svm#wine2#instance:86#num_features:11#feat_selec:lasso_path#discre:decile#sai:False#kernel_s:6.html
serialized_explanation/svm#wine2#instance:86#num_features:11#feat_selec:none#discre:quartile#sai:True#kernel_s:6.html
serialized_explanation/svm#wine2#instance:86#num_features:11#feat_selec:none#discre:quartile#sai:False#kernel_s:6.html
serialized_explanation/svm#wine2#instance:86#num_features:11#feat_selec:none#discre:decile#sai:True#kernel_s:6.html
serialized_explanation/svm#wine2#instance:86#num_features:11#feat_selec:none#discre:decile#sai:False#kernel_s:6.html
serialized_explanation/svm#wine2#instance:86#num_features:11#feat_selec:auto#discre:quartile#sai:True#kernel_s:6.html
serialized_explanation/svm#wine2#instance:86#num_features:11#feat_selec:auto#discre:quartile#sai:False#kernel_s:6.html
serialized_explanation/svm#wine2#instance:86#num

serialized_explanation/svm#wine2#instance:86#num_features:11#feat_selec:none#discre:quartile#sai:False#kernel_s:9.html
serialized_explanation/svm#wine2#instance:86#num_features:11#feat_selec:none#discre:decile#sai:True#kernel_s:9.html
serialized_explanation/svm#wine2#instance:86#num_features:11#feat_selec:none#discre:decile#sai:False#kernel_s:9.html
serialized_explanation/svm#wine2#instance:86#num_features:11#feat_selec:auto#discre:quartile#sai:True#kernel_s:9.html
serialized_explanation/svm#wine2#instance:86#num_features:11#feat_selec:auto#discre:quartile#sai:False#kernel_s:9.html
serialized_explanation/svm#wine2#instance:86#num_features:11#feat_selec:auto#discre:decile#sai:True#kernel_s:9.html
serialized_explanation/svm#wine2#instance:86#num_features:11#feat_selec:auto#discre:decile#sai:False#kernel_s:9.html
serialized_explanation/svm#wine2#instance:1458#num_features:11#feat_selec:forward_selection#discre:quartile#sai:True#kernel_s:9.html
serialized_explanation/svm#wine2#instance:145

### Diabetes dataset

In [27]:
# import numpy as np
import lime
import lime.lime_tabular
import sys
sys.path.append('./ml_helper')
import ml_helper_train_model as model_train
import ml_helper_serialize_model as serializer
sys.path.append('./explanation_helper')
import expla_helper as explanation
from sklearn.model_selection import train_test_split

diabetes = pd.read_csv('data/diabetes_limpo.csv', sep = ',')
# diabetes.drop(['Unnamed: 0'], axis = 1, inplace = True)
diabetes = explanation.to_float(diabetes)

model_file_name = 'serialized_model/svm_diabetes2.pkl'
model = serializer.load_model(model_file_name)

dataset = diabetes.copy(deep = True)
target_names = np.array(['saudável', 'doente'])

folder = 'serialized_explanation/'
model_dataset = model_file_name.replace('serialized_model/','').replace('.pkl', '').replace('_', '#')


kernel_width = [math.ceil(dataset.shape[1]*0.1),
                math.ceil(dataset.shape[1]*0.3),
                math.ceil(dataset.shape[1]*0.5),
                math.ceil(dataset.shape[1]*0.7)]

for size in kernel_width:
    for sample in diabetes_sample:
        for f_selec in feature_selection:
            for disc in discretizer:
                for sai in sample_around_instance:
                    instance_index = sample
                    num_features = dataset.columns.size - 1 #removing the target

                    path = """{0}{1}#instance:{2}#""".format(folder, model_dataset, instance_index)
                    params_01 = """num_features:{0}#feat_selec:{1}#discre:{2}#""".format(num_features, f_selec, disc)
                    params_02 = """sai:{0}#kernel_s:{1}.html""".format(sai, size)
                    
                    final_file = """{0}{1}{2}""".format(path, params_01, params_02)
                    print(final_file)

                    explanation.explain_it_param(model,
                                               dataset,
                                               target_names,
                                               num_features,
                                               instance_index,
                                               final_file,
                                               #object level
                                               size,
                                               f_selec,     
                                               discretize_continuous,
                                               disc,
                                               sai)


serialized_explanation/svm#diabetes2#instance:291#num_features:8#feat_selec:forward_selection#discre:quartile#sai:True#kernel_s:1.html
serialized_explanation/svm#diabetes2#instance:291#num_features:8#feat_selec:forward_selection#discre:quartile#sai:False#kernel_s:1.html
serialized_explanation/svm#diabetes2#instance:291#num_features:8#feat_selec:forward_selection#discre:decile#sai:True#kernel_s:1.html
serialized_explanation/svm#diabetes2#instance:291#num_features:8#feat_selec:forward_selection#discre:decile#sai:False#kernel_s:1.html
serialized_explanation/svm#diabetes2#instance:291#num_features:8#feat_selec:lasso_path#discre:quartile#sai:True#kernel_s:1.html
serialized_explanation/svm#diabetes2#instance:291#num_features:8#feat_selec:lasso_path#discre:quartile#sai:False#kernel_s:1.html
serialized_explanation/svm#diabetes2#instance:291#num_features:8#feat_selec:lasso_path#discre:decile#sai:True#kernel_s:1.html
serialized_explanation/svm#diabetes2#instance:291#num_features:8#feat_selec:las

serialized_explanation/svm#diabetes2#instance:291#num_features:8#feat_selec:forward_selection#discre:quartile#sai:False#kernel_s:3.html
serialized_explanation/svm#diabetes2#instance:291#num_features:8#feat_selec:forward_selection#discre:decile#sai:True#kernel_s:3.html
serialized_explanation/svm#diabetes2#instance:291#num_features:8#feat_selec:forward_selection#discre:decile#sai:False#kernel_s:3.html
serialized_explanation/svm#diabetes2#instance:291#num_features:8#feat_selec:lasso_path#discre:quartile#sai:True#kernel_s:3.html
serialized_explanation/svm#diabetes2#instance:291#num_features:8#feat_selec:lasso_path#discre:quartile#sai:False#kernel_s:3.html
serialized_explanation/svm#diabetes2#instance:291#num_features:8#feat_selec:lasso_path#discre:decile#sai:True#kernel_s:3.html
serialized_explanation/svm#diabetes2#instance:291#num_features:8#feat_selec:lasso_path#discre:decile#sai:False#kernel_s:3.html
serialized_explanation/svm#diabetes2#instance:291#num_features:8#feat_selec:none#discre

serialized_explanation/svm#diabetes2#instance:291#num_features:8#feat_selec:forward_selection#discre:decile#sai:True#kernel_s:5.html
serialized_explanation/svm#diabetes2#instance:291#num_features:8#feat_selec:forward_selection#discre:decile#sai:False#kernel_s:5.html
serialized_explanation/svm#diabetes2#instance:291#num_features:8#feat_selec:lasso_path#discre:quartile#sai:True#kernel_s:5.html
serialized_explanation/svm#diabetes2#instance:291#num_features:8#feat_selec:lasso_path#discre:quartile#sai:False#kernel_s:5.html
serialized_explanation/svm#diabetes2#instance:291#num_features:8#feat_selec:lasso_path#discre:decile#sai:True#kernel_s:5.html
serialized_explanation/svm#diabetes2#instance:291#num_features:8#feat_selec:lasso_path#discre:decile#sai:False#kernel_s:5.html
serialized_explanation/svm#diabetes2#instance:291#num_features:8#feat_selec:none#discre:quartile#sai:True#kernel_s:5.html
serialized_explanation/svm#diabetes2#instance:291#num_features:8#feat_selec:none#discre:quartile#sai:

serialized_explanation/svm#diabetes2#instance:291#num_features:8#feat_selec:forward_selection#discre:decile#sai:False#kernel_s:7.html
serialized_explanation/svm#diabetes2#instance:291#num_features:8#feat_selec:lasso_path#discre:quartile#sai:True#kernel_s:7.html
serialized_explanation/svm#diabetes2#instance:291#num_features:8#feat_selec:lasso_path#discre:quartile#sai:False#kernel_s:7.html
serialized_explanation/svm#diabetes2#instance:291#num_features:8#feat_selec:lasso_path#discre:decile#sai:True#kernel_s:7.html
serialized_explanation/svm#diabetes2#instance:291#num_features:8#feat_selec:lasso_path#discre:decile#sai:False#kernel_s:7.html
serialized_explanation/svm#diabetes2#instance:291#num_features:8#feat_selec:none#discre:quartile#sai:True#kernel_s:7.html
serialized_explanation/svm#diabetes2#instance:291#num_features:8#feat_selec:none#discre:quartile#sai:False#kernel_s:7.html
serialized_explanation/svm#diabetes2#instance:291#num_features:8#feat_selec:none#discre:decile#sai:True#kernel_